In [ ]:
# 개선된 췌장 관련 텍스트 추출 및 분류 시스템
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import os
import chardet
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# ================================================================
# 1. 텍스트 전처리 및 췌장 관련 내용 추출 함수
# ================================================================

def extract_pancreas_related_text(text, is_findings=False):
    """
    텍스트에서 췌장 관련 내용만 추출하고 다른 장기 관련 내용을 제거하는 함수

    Args:
        text: 원본 텍스트
        is_findings: 검사결과 컬럼인지 여부 (True: 검사결과, False: 결론)

    Returns:
        췌장 관련 내용만 포함된 문자열
    """
    # None 또는 비문자열 입력 처리
    if text is None or not isinstance(text, str):
        return "" if text is None else str(text)

    # 결과 저장용 리스트
    pancreas_content = []

    # 1. 텍스트 정리 및 준비
    text = text.replace("\r", " ").replace("\n\n", "\n")

    # 2. 텍스트를 의미 있는 단위로 분리

    # 2-1. 번호 항목 추출 (예: "3. ..." 또는 "3. ...\n    -- R/O ...")
    numbered_items = re.findall(r'\d+\.\s+[^\n]+(?:\n\s+(?:--|;|-->|→)[^\n]+)?', text)

    # 2-2. 번호 항목을 제외한 나머지 텍스트를 문장 단위로 분리
    # 번호 항목 제거
    remaining_text = text
    for item in numbered_items:
        remaining_text = remaining_text.replace(item, '')

    # 남은 텍스트를 문장 단위로 분리 (마침표, 개행 등 기준)
    sentences = re.split(r'(?<=[.!?])\s+|\n', remaining_text)

    # 3. 췌장 관련 키워드 정의
    pancreas_keywords = [
        'pancreas', 'pancreatic', 'p-duct', 'wirsung', 'santorini', 'uncinate',
        'tail portion', 'head portion', 'body portion', 'neck portion',
        'ipmn', 'intraductal papillary',
        '췌장', '췌관', '체부', '미부', '두부', '구상돌기'
    ]

    # 다른 장기 키워드 (부정적 키워드)
    other_organs = [
        'liver', 'hepatic', 'kidney', 'renal', 'spleen', 'gallbladder', 'gb',
        'bile duct', 'biliary', 'bowel', 'colon', 'adrenal', 'ureter', 'bladder',
        'uterus', 'ovary', 'prostate', 'appendix', 'stomach', 'bosniak',
        '간', '신장', '비장', '담낭', '담도', '방광', '요관', '림프절', '대장', '소장',
        '자궁', '난소', '전립선', '충수', '위'
    ]

    # 4. 번호 항목 중 췌장 관련 항목 추출
    for item in numbered_items:
        has_pancreas = any(re.search(r'\b' + re.escape(kw) + r'\b', item, re.I) for kw in pancreas_keywords)

        if has_pancreas:
            # 다른 장기만 언급된 부분 제거
            # 여기서는 다른 장기 중심의 문장이 아닌 경우에만 포함
            has_other_organ_focus = False

            # 다른 장기가 주요 주제인지 확인
            for organ in other_organs:
                # 다른 장기가 문장 시작 부분에 있는지 확인 (주요 주제)
                if re.search(r'\d+\.\s+[^.]*\b' + re.escape(organ) + r'\b', item, re.I):
                    has_other_organ_focus = True
                    break

            if not has_other_organ_focus:
                # 부정어구 처리 ("No change" 등)
                if re.search(r'no\s+(?:interval\s+)?change', item.lower(), re.I):
                    # "No change" 표현은 변화가 없다는 의미 (존재함)
                    item = re.sub(
                        r'(no\s+(?:interval\s+)?change\s+of\s+)(.*?)(?=\.|$)',
                        r'\1\2 [UNCHANGED_EXISTS]',
                        item,
                        flags=re.I
                    )

                # 감별진단(R/O) 패턴 처리
                if re.search(r'(?:--|;|-->|→)\s*(?:R/O|DDx)', item.lower(), re.I):
                    if re.search(r'cyst|cystic', item.lower(), re.I):
                        # 낭종이 감별진단에 포함됨 (존재 가능성)
                        item = re.sub(
                            r'((?:--|;|-->|→)\s*(?:R/O|DDx).*?(?:cyst|cystic).*?)(?=\.|$)',
                            r'\1 [CYST_POSSIBLE]',
                            item,
                            flags=re.I
                        )

                pancreas_content.append(item.strip())

    # 5. 문장 중 췌장 관련 문장 추출
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        # 췌장 관련 키워드가 있는 문장만 추출
        has_pancreas = any(re.search(r'\b' + re.escape(kw) + r'\b', sentence, re.I) for kw in pancreas_keywords)

        if has_pancreas:
            # 다른 장기 언급이 있는지 확인
            has_other_organ = any(re.search(r'\b' + re.escape(organ) + r'\b', sentence, re.I) for organ in other_organs)

            if has_other_organ:
                # 다른 장기가 주요 주제인지 확인
                has_other_organ_focus = False

                for organ in other_organs:
                    # 다른 장기가 문장 시작 부분에 있는지 확인 (주요 주제)
                    if re.search(r'^[^.]*\b' + re.escape(organ) + r'\b', sentence, re.I):
                        has_other_organ_focus = True
                        break

                if not has_other_organ_focus:
                    # 췌장 중심 문장만 포함
                    pancreas_content.append(sentence.strip())
            else:
                # 다른 장기 언급이 없는 췌장 관련 문장은 그대로 추가
                pancreas_content.append(sentence.strip())

    # 6. 특수 케이스 처리

    # 6-1. 검사 제목이 PANCREAS CT인 경우 추가
    pancreas_ct_pattern = re.compile(r'PANCREAS\s+CT', re.I)
    if pancreas_ct_pattern.search(text):
        ct_lines = [line for line in text.split('\n') if pancreas_ct_pattern.search(line)]
        if ct_lines:
            pancreas_content.insert(0, ct_lines[0].strip())

    # 6-2. "No change" + 췌장 패턴 추가 검사
    no_change_pattern = re.compile(r'no\s+(?:interval\s+)?change\s+of.*?pancreas', re.I)
    if no_change_pattern.search(text):
        for match in no_change_pattern.finditer(text):
            no_change_text = match.group(0)
            if no_change_text not in ' '.join(pancreas_content):
                pancreas_content.append(no_change_text.strip())

    # 6-3. "R/O" + 췌장 관련 내용 추가 검사
    ro_pattern = re.compile(r'(?:--|;|-->|→)\s*(?:R/O|DDx)[^.]*?pancreas', re.I)
    if ro_pattern.search(text):
        for match in ro_pattern.finditer(text):
            ro_text = match.group(0)
            if ro_text not in ' '.join(pancreas_content):
                pancreas_content.append(ro_text.strip())

    # 6-4. 낭종 관련 명확한 패턴 추가 검사
    cyst_patterns = [
        r'pancreatic\s+cyst', r'cyst\s+in\s+pancreas', r'cystic\s+lesion.*?pancreas',
        r'췌장\s+낭종', r'낭종.*?췌장', r'cystic\s+lesion,\s*pancreas', r'cysts?,\s*pancreas'
    ]

    for pattern in cyst_patterns:
        if re.search(pattern, text, re.I):
            for match in re.finditer(pattern, text, re.I):
                cyst_text = text[max(0, match.start() - 50):min(len(text), match.end() + 50)]
                # 문장 경계 조정
                cyst_text = re.sub(r'^[^.]*?\.', '', cyst_text)
                cyst_text = re.sub(r'\.[^.]*$', '', cyst_text)

                if cyst_text.strip() and cyst_text not in ' '.join(pancreas_content):
                    pancreas_content.append(cyst_text.strip())

    # 7. 결과 조합 및 중복 제거
    unique_content = []
    for item in pancreas_content:
        if item and item not in unique_content:
            unique_content.append(item)

    # 8. 특별 처리: 텍스트가 너무 길거나 복잡한 경우 간소화 (필요시)
    if is_findings and len(' '.join(unique_content)) > 1000:  # 검사결과가 너무 길면
        # 결론적인 표현만 유지
        conclusion_keywords = ['conclusion', 'impression', 'summary', '결론', '판독소견']
        conclusion_content = []

        for item in unique_content:
            if any(keyword in item.lower() for keyword in conclusion_keywords):
                conclusion_content.append(item)

        if conclusion_content:
            unique_content = conclusion_content

    # 결과 반환
    result = ' '.join(unique_content)

    # 공백 정리
    result = re.sub(r'\s+', ' ', result).strip()

    return result

def remove_all_other_organ_mentions(text):
    """
    모든 다른 장기 언급을 제거하는 함수 - 췌장 관련 텍스트만 유지

    Args:
        text: 원본 텍스트

    Returns:
        다른 장기 언급이 제거된 텍스트
    """
    # None 또는 비문자열 입력 처리
    if text is None or not isinstance(text, str):
        return "" if text is None else str(text)

    # 결과 저장용 문장 리스트
    cleaned_sentences = []

    # 문장 단위로 분리
    sentences = re.split(r'(?<=[.!?])\s+|\n', text)

    # 췌장 관련 키워드
    pancreas_keywords = [
        'pancreas', 'pancreatic', 'p-duct', 'wirsung', 'santorini', 'uncinate',
        'tail portion', 'head portion', 'body portion', 'neck portion',
        'ipmn', 'intraductal papillary',
        '췌장', '췌관', '체부', '미부', '두부', '구상돌기'
    ]

    # 다른 장기 키워드
    other_organs = [
        'liver', 'hepatic', 'kidney', 'renal', 'spleen', 'gallbladder', 'gb',
        'bile duct', 'biliary', 'bowel', 'colon', 'adrenal', 'ureter', 'bladder',
        'uterus', 'ovary', 'prostate', 'appendix', 'stomach', 'bosniak',
        '간', '신장', '비장', '담낭', '담도', '방광', '요관', '림프절', '대장', '소장',
        '자궁', '난소', '전립선', '충수', '위'
    ]

    # 각 문장 처리
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        # 췌장 관련 키워드가 있는 문장만 유지
        has_pancreas = any(re.search(r'\b' + re.escape(kw) + r'\b', sentence, re.I) for kw in pancreas_keywords)

        if has_pancreas:
            # 다른 장기 언급이 없거나, 주요 주제가 췌장인 경우만 유지
            has_other_organ_focus = False

            for organ in other_organs:
                # 다른 장기가 문장 시작 부분에 있는지 확인 (주요 주제)
                if re.search(r'^[^.]*\b' + re.escape(organ) + r'\b', sentence, re.I):
                    has_other_organ_focus = True
                    break

            if not has_other_organ_focus:
                # 다른 장기 언급 부분 제거
                cleaned_sentence = sentence
                for organ in other_organs:
                    # 해당 장기 언급 부분만 제거 (문장 전체를 버리지 않음)
                    organ_pattern = r'[^,.;:]*\b' + re.escape(organ) + r'\b[^,.;:]*[,.;:]?\s*'
                    cleaned_sentence = re.sub(organ_pattern, '', cleaned_sentence, flags=re.I)

                # 정제 후에도 췌장 언급이 남아있는지 확인
                if any(re.search(r'\b' + re.escape(kw) + r'\b', cleaned_sentence, re.I) for kw in pancreas_keywords):
                    cleaned_sentences.append(cleaned_sentence.strip())

    # 결과 조합
    result = ' '.join(cleaned_sentences)

    # 공백 정리
    result = re.sub(r'\s+', ' ', result).strip()

    return result

def process_medical_context(text):
    """
    의학적 맥락을 고려한 텍스트 처리

    Args:
        text: 처리할 텍스트

    Returns:
        처리된 텍스트
    """
    # None 또는 비문자열 입력 처리
    if text is None or not isinstance(text, str):
        return "" if text is None else str(text)

    processed_text = text

    # 1. "No change" 표현 처리 - 변화 없음은 존재함을 의미
    no_change_patterns = [
        r'no\s+(?:interval\s+)?change\s+of\s+(.*?)(?=\.|$)',
        r'no\s+(?:interval\s+)?change\s+in\s+(.*?)(?=\.|$)'
    ]

    for pattern in no_change_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            original = f"No change of {match}"
            replacement = f"UNCHANGED: {match} (STILL PRESENT)"
            processed_text = processed_text.replace(original, replacement)

    # 2. "R/O" 표현 처리 - 배제하는 것이 아니라 고려함을 의미
    ro_patterns = [
        r'R/O\s+(.*?)(?=\.|$)',
        r'(?:--|;|-->|→)\s*R/O\s+(.*?)(?=\.|$)',
        r'DDx\s*\)?\s*(.*?)(?=\.|$)'
    ]

    for pattern in ro_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            if 'cyst' in match.lower() or 'cystic' in match.lower():
                original = f"R/O {match}"
                replacement = f"CONSIDERING: {match} (POTENTIALLY PRESENT)"
                processed_text = processed_text.replace(original, replacement)

    # 3. 명확한 부정어 처리 - 진짜 없는 것
    negation_patterns = [
        r'not\s+(?:definite|detected|visualized|seen)',
        r'normal\s+pancreas',
        r'unremarkable\s+pancreas',
        r'no\s+abnormality\s+in\s+pancreas',
        r'정상\s+췌장',
        r'췌장\s+정상'
    ]

    for pattern in negation_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            # 진짜 부정어 표시
            processed_text = "TRUE NEGATION: " + processed_text
            break

    return processed_text

def read_with_correct_encoding(file_path):
    """
    파일의 인코딩을 자동 감지하여 올바르게 읽는 함수

    Args:
        file_path: 파일 경로

    Returns:
        로드된 DataFrame
    """
    # 파일이 존재하는지 확인
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")

    # 인코딩 감지
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # 처음 10000 바이트만 읽기
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    print(f"파일 '{file_path}'의 감지된 인코딩: {encoding} (신뢰도: {result['confidence']})")

    # 다양한 인코딩 시도
    encodings_to_try = [encoding, 'utf-8', 'cp949', 'euc-kr', 'latin1']

    for enc in encodings_to_try:
        try:
            return pd.read_csv(file_path, encoding=enc)
        except UnicodeDecodeError:
            print(f"인코딩 {enc}로 파일을 읽는 데 실패했습니다. 다른 인코딩 시도...")
        except Exception as e:
            print(f"파일 읽기 오류: {e}")
            break

    # 모든 인코딩 시도 실패 시
    raise ValueError(f"파일 '{file_path}'을 읽을 수 없습니다. 지원되는 인코딩으로 저장하세요.")

# ================================================================
# 2. ImprovedPancreaticCystBinaryClassifier 클래스 정의
# ================================================================

class ImprovedPancreaticCystBinaryClassifier:
    # 클래스 상수 정의
    PCL_CLASS = 0          # 췌장 낭종 (Pancreatic Cystic Lesion)
    NON_PCL_CLASS = 1      # 비췌장 낭종 (Non-Pancreatic Cystic Lesion)
    UNCERTAIN_CLASS = 2    # 불확실 케이스 (요 방사선과 의사 검토 필요)

    # 결과 영역(zone) 상수 정의
    STANDARD_ZONE = "standard"    # 일반 케이스
    UNCERTAIN_ZONE = "uncertain"  # 불확실 케이스

    def __init__(self, biobert_path="dmis-lab/biobert-v1.1"):
        """초기화 - 개선된 버전"""
        print("Initializing BioBERT model...")
        # 클래스 레이블 설명 출력
        print(f"Class labels: {self.PCL_CLASS}=PCL, {self.NON_PCL_CLASS}=Non-PCL, {self.UNCERTAIN_CLASS}=Uncertain")

        # BioBERT 모델 초기화
        try:
            self.biobert_model, self.tokenizer = self._create_biobert_with_pattern_awareness(biobert_path)
        except Exception as e:
            print(f"Error initializing BioBERT: {e}")
            print("Continuing with limited functionality...")
            self.biobert_model, self.tokenizer = None, None

        # 패턴 분류기 초기화 (RandomForest)
        self.pattern_classifier = RandomForestClassifier(
            n_estimators=100,
            class_weight={self.PCL_CLASS: 1.8, self.NON_PCL_CLASS: 0.5},  # PCL 클래스에 더 높은 가중치
            random_state=42
        )

        # 가중치 조정: BioBERT vs 패턴 매칭 가중치
        self.weight_biobert = 0.6
        self.weight_pattern = 0.4

        # 결론과 검사결과 컬럼 가중치
        self.conclusion_weight = 0.7  # 결론 가중치 70%
        self.findings_weight = 0.3   # 검사결과 가중치 30%

        # 임계값 조정 - 더 많은 낭종을 잡아내도록 높여줌
        self.cyst_threshold = 0.55  # 0.42에서 0.55로 증가

        # 키워드 중요도 가중치 - 분석 기반 조정
        self.keyword_weights = {
            # 낭종 관련 긍정 키워드 - 가중치 증가
            'cystic lesion': 5.0,     # 값 상향 조정
            'pancreatic cyst': 5.0,
            'pancreatic cysts': 5.0,
            'pancreas cyst': 5.0,
            'pancreas cysts': 5.0,
            'cyst in pancreas': 5.0,
            'cysts in pancreas': 5.0,
            'tiny cyst': 4.0,         # 값 상향 조정
            'small cyst': 4.0,        # 값 상향 조정
            'ipmn': 5.0,
            'branch duct': 3.5,
            'branch duct type': 3.5,
            'main duct': 3.5,
            'main duct type': 3.5,
            'r/o ipmn': 3.5,
            'p-duct dilatation': 3.5,
            'pancreatic duct dilatation': 3.5,
            'duct dilation': 3.5,
            'wirsung duct': 3.5,        # 주췌관 관련 추가
            'santorini duct': 3.0,      # 부췌관 관련 추가
            'intraductal papillary': 4.0, # IPMN 관련 추가
            'side branch': 3.0,         # 분지 관련 추가
            'multiseptated': 2.0,
            'lobulated': 2.0,
            'septated': 2.0,

            # 쉼표 구조 패턴 추가 - 매우 중요한 패턴이므로 가중치 상향
            'cystic lesion, pancreas': 5.0,   # 값 상향 조정
            'cystic lesions, pancreas': 5.0,  # 값 상향 조정
            'cyst, pancreas': 5.0,           # 값 상향 조정
            'cysts, pancreas': 5.0,          # 값 상향 조정
            'several cysts, pancreas': 5.0,
            'multiple cysts, pancreas': 5.0,
            'few cysts, pancreas': 5.0,
            'two cysts, pancreas': 5.0,
            'three cysts, pancreas': 5.0,
            'several tiny cysts': 5.0,       # 추가
            'several small cysts': 5.0,      # 추가
            'multiple tiny cysts': 5.0,      # 추가
            'multiple small cysts': 5.0,     # 추가

            # 변화 없음 패턴 - 강화
            'no change': 2.0,
            'no change of pancreatic cyst': 4.5,  # 값 상향 조정
            'no interval change': 2.0,
            'no interval change of pancreatic cyst': 4.5,  # 값 상향 조정
            'interval change': 1.0,
            'no change of low density lesion': 2.5,
            'no interval change of low density lesion': 2.5,
            'no change of cystic lesion': 4.5,  # 추가
            'no interval change of cystic lesion': 4.5,  # 추가

            # 비낭종 관련 키워드 - 부정어 처리 강화
            'normal pancreas': -5.0,   # 값 상향 조정 (더 강한 부정)
            'unremarkable pancreas': -5.0,  # 값 상향 조정
            'unremarkable finding in pancreas': -5.0,  # 값 상향 조정
            'no abnormality in pancreas': -5.0,  # 값 상향 조정
            'no definite detectable': -3.0,  # 값 상향 조정
            'not definite': -5.0,  # 추가
            'not detected': -5.0,  # 추가
            'not visualized': -5.0,  # 추가
            'not seen': -5.0,  # 추가
            'cancer': -2.5,
            'adenocarcinoma': -2.5,
            'malignancy': -2.5,
            'tumor': -2.0,
            'mass': -1.8,
            'solid': -1.5,
            'intraparenchymal fat': -1.5,
            'fatty infiltration': -1.5,
            'fatty liver': -1.0,
            'lipoma': -2.0,
            'calcification': -1.0,
            'enhancing nodule': -2.0,
            'enhancing wall': -1.5,
            'urothelial carcinoma': -2.5,
            'rcc': -2.5,
            'hcc': -2.5,

            # 위치 정보 (최소 가중치)
            'pancreas head': 0.3,
            'pancreas tail': 0.3,
            'pancreas body': 0.3,
            'pancreatic head': 0.3,
            'pancreatic tail': 0.3,
            'pancreatic body': 0.3,
            'uncinate process': 0.3,

            # 주췌관 확장 관련 패턴
            'minimal dilatation of pancreatic duct': 2.0,
            'mild dilatation of pancreatic duct': 2.5,
            'moderate dilatation of pancreatic duct': 3.0,
            'dilatation of main pancreatic duct': 3.5,
            'dilatation of pancreatic duct': 3.0,

            # 양성 저음영 병변 패턴
            'benign low density lesion in pancreas': 2.5,
            'benign looking low density lesion in pancreas': 2.5,
            'benign low attenuation lesion in pancreas': 2.5,
            'benign low density lesion, pancreas': 2.5,
            'benign low attenuation lesion, pancreas': 2.5,

            # 경계 케이스 표현
            'r/o benign': 1.0,
            'too small to characterize': 0.2,
            'ddx': 0.0,
            'focal fat': -1.0,
            'not delineation': 0.5,
            'vs fat': 0.0,
            'vs lipoma': 0.0,
            'vs infiltration': 0.0,
            'or fat': 0.0,
            'or lipoma': 0.0,
            'or infiltration': 0.0,
            'remnant pancreas': 1.5,  # 잔여 췌장 키워드 추가
            'senile change': -1.5,
            'aging process': -1.5,

            # 다른 장기 키워드 (감점)
            'renal cyst': -5.0,  # 값 상향 조정 (더 강한 부정)
            'kidney cyst': -5.0,  # 값 상향 조정
            'bosniak': -5.0,  # 값 상향 조정
            'liver cyst': -5.0,  # 값 상향 조정
            'hepatic cyst': -5.0,  # 값 상향 조정
            'spleen cyst': -5.0,  # 값 상향 조정
            'gb cyst': -5.0,  # 값 상향 조정
            'gallbladder cyst': -5.0,  # 값 상향 조정

            # 한글 키워드 추가
            '췌장': 3.0,
            '췌장 낭종': 5.0,
            '낭종': 3.0,
            '낭성': 2.5,
            '낭성 병변': 3.0,
            '췌관': 2.5,
            '주췌관': 3.0,
            '부췌관': 2.5,
            '체부': 0.3,
            '미부': 0.3,
            '두부': 0.3,
            '정상 췌장': -5.0,  # 값 상향 조정
            '췌장 정상': -5.0,  # 값 상향 조정
        }

        # 장기 컨텍스트 가중치
        self.organ_context = {
            'positive': ['pancreas', 'pancreatic', 'wirsung', 'santorini', 'uncinate', '췌장', '췌관'],
            'negative': ['liver', 'hepatic', 'renal', 'kidney', 'spleen', 'gb', 'gallbladder',
                         '간', '신장', '비장', '담낭', '담도']
        }

        # 부정어 목록 - 명확하게 확장
        self.negation_terms = [
            'no', 'not', 'without', 'absent', 'negative', 'unremarkable', 'normal',
            'no evidence', 'no definite', 'no detectable', 'no significant', 'no abnormality',
            'no demonstrable', 'no visible', 'not seen', 'not identified', 'not detected',
            'not visualized', 'not definite', 'not delineated', 'not remarkable',
            '없음', '정상', '관찰되지 않음'
        ]

        # 감별진단 컨텍스트 분석 (R/O 패턴) - 추가
        self.differential_diagnosis_terms = [
            'r/o', 'ddx', 'differential diagnosis', 'r-o', 'rule out', 'versus', 'vs.', 'vs',
            'suspicious', 'suspicious for', 'suggesting', 'suggestive of', 'likely', 'possible',
            'consider', 'suspected', 'suspicion of', 'cannot exclude', 'cannot rule out',
            'most likely', 'probable', 'consistent with'
        ]

        # 회색 영역 분류 패턴 - raw string 사용
        self.gray_zone_a_pattern = re.compile(r'(?:cystic|cyst).*(?:(?:r/o|ddx|vs|or|versus).*(?:fat|infiltration|lipoma)|(?:fat|infiltration|lipoma).*(?:r/o|ddx|vs|or|versus))', re.I)
        self.gray_zone_b_pattern = re.compile(r'(?:low\s+(?:density|attenuation)|(?:fat|fatty)).*(?:lesion|nodule)(?!.*\bcyst)', re.I)

        # "or" 패턴 불확실 케이스 패턴 - raw string 사용
        self.or_pattern = re.compile(
            r'(?:cyst|cystic).*?(?:or|vs|versus).*?(?:fat|lipoma|infiltration)|'
            r'(?:fat|lipoma|infiltration).*?(?:or|vs|versus).*?(?:cyst|cystic)',
            re.I
        )

        # 감별진단(DDx) 패턴 - raw string 사용
        self.ddx_pattern = re.compile(
            r'ddx\s*\)?\s*(?:tiny|small)?\s*(?:cystic|cyst).*?(?:vs|versus).*?(?:fat|peripancreatic)|'
            r'ddx\s*\)?\s*(?:fat|peripancreatic).*?(?:vs|versus).*?(?:cystic|cyst)',
            re.I
        )

        # Not delineation 패턴 - raw string 사용
        self.not_delineation_pattern = re.compile(
            r'not\s+delineation\s+of.*?(?:low\s+density|low\s+attenuation)\s+lesion',
            re.I
        )

        # 주췌관 확장 패턴 - raw string 사용
        self.pancreatic_duct_dilatation_pattern = re.compile(
            r'(?:minimal|mild|moderate)?\s*(?:dilatation|dilation)\s+of\s+(?:main\s+)?pancreatic\s+duct',
            re.I
        )

        # 주췌관 확장 + IPMN 패턴 - raw string 사용
        self.duct_ipmn_pattern = re.compile(
            r'(?:dilatation|dilation)\s+of\s+(?:main\s+)?pancreatic\s+duct.*?(?:ipmn|intraductal\s+papillary|mucinous)',
            re.I
        )

        # "No change" + 저음영 병변 패턴 - raw string 사용
        self.no_change_low_density_pattern = re.compile(
            r'no\s+(?:interval\s+)?change\s+of.*?(?:low\s+density|low\s+attenuation)\s+lesions?\s+in\s+(?:the\s+)?pancreas',
            re.I
        )

        # 양성 저음영 병변 패턴 - raw string 사용
        self.benign_low_density_pattern = re.compile(
            r'benign(?:\s+looking)?\s+(?:low\s+density|low\s+attenuation)\s+lesions?\s+in\s+(?:the\s+)?pancreas|'
            r'benign(?:\s+looking)?\s+(?:low\s+density|low\s+attenuation)\s+lesions?\s*,\s*pancreas',
            re.I
        )

        # 지방 패턴 - raw string 사용
        self.fat_pattern = re.compile(
            r'(?:fat|fatty|lipoma|fat\s+infiltration|fatty\s+infiltration|invaginated\s+fat)\s+in\s+(?:the\s+)?pancreas',
            re.I
        )

        # 노화 관련 표현 - raw string 사용
        self.senile_pattern = re.compile(
            r'senile\s+change|aging\s+process',
            re.I
        )

        # 잔여 췌장 패턴 - raw string 사용
        self.remnant_pattern = re.compile(
            r'remnant\s+pancreas',
            re.I
        )

        # 부정어 패턴 - 명확한 정의 (raw string 사용)
        self.negation_patterns = [
            re.compile(r'no\s+(?:definite|remarkable|significant)?\s*(?:abnormality|change)\s+(?:of|in)\s+(?:the\s+)?pancreas', re.I),
            re.compile(r'not\s+(?:definite|detected|visualized|seen)(?:\s+in\s+this\s+study)?', re.I),
            re.compile(r'normal\s+pancreas', re.I),
            re.compile(r'unremarkable\s+(?:finding\s+in\s+)?pancreas', re.I),
            re.compile(r'정상\s+췌장', re.I),
            re.compile(r'췌장\s+정상', re.I)
        ]

        # 명확한 췌장 낭종 패턴 - 훨씬 더 포괄적으로 정의 (raw string 사용)
        self.clear_pancreatic_cyst_patterns = [
            re.compile(r'pancreatic\s+cysts?', re.I),
            re.compile(r'cysts?\s+(?:in|of)\s+(?:the\s+)?pancreas', re.I),
            re.compile(r'(?:cystic\s+lesions?|cysts?)\s+(?:in|of|,)\s+(?:the\s+)?pancreas', re.I),
            re.compile(r'(?:several|multiple|few|tiny|small)\s+(?:cystic\s+lesions?|cysts?)\s+(?:in|of|,)\s+(?:the\s+)?pancreas', re.I),
            re.compile(r'(?:several|multiple|few|tiny|small)\s+pancreatic\s+cysts?', re.I),
            re.compile(r'cysts?\s+(?:in|of)\s+(?:the\s+)?(?:head|body|tail|uncinate)\s+(?:portion\s+)?(?:of\s+)?(?:the\s+)?pancreas', re.I),
            re.compile(r'(?:several|multiple|few|tiny|small)\s+cysts?\s+(?:in|of)\s+(?:the\s+)?(?:head|body|tail|uncinate)\s+(?:portion\s+)?(?:of\s+)?(?:the\s+)?pancreas', re.I),
            re.compile(r'ipmn', re.I),
            re.compile(r'intraductal\s+papillary\s+mucinous', re.I),
            re.compile(r'췌장\s*낭종', re.I),
            re.compile(r'췌장.*?낭성.*?병변', re.I),
            re.compile(r'낭종.*?췌장', re.I),
            re.compile(r'cystic\s+lesions?,\s*pancreas', re.I),
            re.compile(r'(?:several|multiple|few|two|three)\s+cysts?,\s*pancreas', re.I),
            re.compile(r'no\s+(?:interval\s+)?change\s+of\s+(?:.*?pancreatic\s+cyst|.*?cyst.*?pancreas)', re.I)
        ]

        print("Classifier initialization complete!")

    def _create_biobert_with_pattern_awareness(self, model_name):
        """개선된 패턴 인식 BioBERT 모델 생성"""
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = TFAutoModelForSequenceClassification.from_pretrained(
                model_name,
                num_labels=2,
                from_pt=True
            )

            # 패턴 인식 강화를 위한 특수 토큰 추가
            special_tokens = {
                'additional_special_tokens': [
                    # 기본 낭종/종양 관련 토큰
                    '[CYST]', '[TUMOR]', '[NORMAL]', '[IPMN]', '[DDXFAT]',

                    # 위치 관련 토큰
                    '[HEAD]', '[BODY]', '[TAIL]', '[UNCIPROC]',

                    # 판독문 특성 관련 토큰
                    '[NEG]', '[OTHERCYST]', '[LIPOMA]', '[CALC]', '[GRAYZONE]',

                    # 명확한 패턴 강조 토큰
                    '[PANCREATICCYST]', '[PDDUCT]',

                    # 추가 토큰 (췌장 특화)
                    '[WIRSUNG]', '[SANTORINI]', '[BRANCH]', '[MAINDUCT]',
                    '[SEPTATION]', '[MULTILOCULAR]', '[LOBULATED]',

                    # 새 패턴 토큰 추가
                    '[COMMAPATTERN]', '[ORPATTERN]', '[DUCDILATION]',
                    '[BENIGNLOW]', '[NOTDELIN]', '[REMNANT]',
                    '[SENILE]', '[DDXPATTERN]', '[NOCHANGE]'
                ]
            }

            # 특수 토큰 추가 및 모델 임베딩 재조정
            added_tokens = tokenizer.add_special_tokens(special_tokens)
            print(f"Added {added_tokens} special tokens to tokenizer")
            model.resize_token_embeddings(len(tokenizer))

            # 토큰 처리 관련 정보 출력
            print(f"Tokenizer vocabulary size: {len(tokenizer)}")
            print(f"Model embedding size: {model.config.vocab_size}")

            # 토큰 할당 테스트
            test_text = "Pancreatic cyst in head. Normal liver."
            encoded = tokenizer.encode(test_text)
            decoded = tokenizer.decode(encoded)
            print(f"Token test - Original: '{test_text}', Decoded: '{decoded}'")

            return model, tokenizer
        except Exception as e:
            print(f"Error creating BioBERT model: {e}")
            return None, None

    def _preprocess_other_organ_cysts(self, text):
        """다른 장기 낭종을 명확한 마커로 전처리"""
        processed = text

        # 다른 장기 + 낭종 패턴에 특수 토큰 마킹
        organ_cyst_patterns = [
            # 간 낭종
            (r'\b(?:liver|hepatic)\s+(?:cyst|cystic)', '[OTHERCYST] [LIVER] $0'),
            # 신장 낭종
            (r'\b(?:renal|kidney|kidneys)\s+(?:cyst|cystic)', '[OTHERCYST] [KIDNEY] $0'),
            # 비장 낭종
            (r'\b(?:spleen)\s+(?:cyst|cystic)', '[OTHERCYST] [SPLEEN] $0'),
            # 담낭 낭종
            (r'\b(?:gb|gallbladder)\s+(?:cyst|cystic)', '[OTHERCYST] [GB] $0'),
            # Bosniak 분류 (신장 낭종 분류)
            (r'\bbosniak', '[OTHERCYST] [KIDNEY] bosniak'),
            # 한글 다른 장기 패턴
            (r'간\s*낭종', '[OTHERCYST] [LIVER] 간 낭종'),
            (r'신장\s*낭종', '[OTHERCYST] [KIDNEY] 신장 낭종'),
            (r'비장\s*낭종', '[OTHERCYST] [SPLEEN] 비장 낭종'),
            (r'담낭\s*낭종', '[OTHERCYST] [GB] 담낭 낭종')
        ]

        # 모든 패턴 적용
        for pattern, replacement in organ_cyst_patterns:
            processed = re.sub(pattern, replacement, processed, flags=re.I)

        # 다른 장기와 낭종이 별도 문장에서 언급된 경우 처리
        organ_patterns = {
            'liver': '[LIVER]',
            'hepatic': '[LIVER]',
            'kidney': '[KIDNEY]',
            'renal': '[KIDNEY]',
            'spleen': '[SPLEEN]',
            'gb': '[GB]',
            'gallbladder': '[GB]',
            '간': '[LIVER]',
            '신장': '[KIDNEY]',
            '비장': '[SPLEEN]',
            '담낭': '[GB]',
            '담도': '[GB]'
        }

        for organ, token in organ_patterns.items():
            processed = re.sub(r'\b' + re.escape(organ) + r'\b', f'{token} {organ}', processed, flags=re.I)

        return processed

    def _preprocess_text_with_pattern_emphasis(self, text):
        """
        텍스트를 패턴 인식을 위해 전처리하고 중요 패턴에 특수 토큰 추가

        Args:
            text: 전처리할 텍스트

        Returns:
            전처리된 텍스트
        """
        # None 또는 비문자열 입력 처리
        if text is None or not isinstance(text, str):
            text = "" if text is None else str(text)

        # 중요 패턴 강조를 위한 특수 토큰 추가
        processed_text = text
        text_lower = text.lower()

        # 췌장 관련 텍스트 필터링 - 다른 장기 관련 문장은 약화시킴
        # 1. 각 입력 텍스트를 문장 단위로 분리
        sentences = re.split(r'[.]\s+|\n', text)

        # 2. 췌장 관련 문장과 다른 장기 관련 문장 분리
        pancreas_sentences = []
        other_organ_sentences = []

        # 췌장 관련 패턴
        pancreas_patterns = [
            r'pancreas', r'pancreatic', r'uncinate', r'췌장', r'췌관'
        ]

        # 다른 장기 패턴
        other_organ_patterns = [
            r'liver', r'hepatic', r'renal', r'kidney', r'spleen', r'gb', r'gallbladder',
            r'bile duct', r'biliary', r'colon', r'intestine', r'stomach', r'adrenal',
            r'ureter', r'bladder', r'prostate', r'uterus', r'ovary',
            r'간', r'신장', r'비장', r'담낭', r'담도', r'장', r'위', r'방광', r'전립선'
        ]

        for sentence in sentences:
            sentence_lower = sentence.lower()

            # 췌장 관련 문장 확인 - raw string을 사용하여 정규식 패턴 개선
            has_pancreas = any(re.search(r'\b' + re.escape(pattern) + r'\b', sentence_lower) for pattern in pancreas_patterns)

            # 다른 장기 관련 문장 확인 - raw string을 사용하여 정규식 패턴 개선
            has_other_organ = any(re.search(r'\b' + re.escape(pattern) + r'\b', sentence_lower) for pattern in other_organ_patterns)

            if has_pancreas:
                pancreas_sentences.append(sentence.strip())
            elif has_other_organ and not has_pancreas:
                # 다른 장기 관련 문장은 무시 (제외)
                other_organ_sentences.append(sentence.strip())

        # 췌장 관련 문장이 있는 경우, 해당 문장들만 사용
        if pancreas_sentences:
            filtered_text = ' '.join(pancreas_sentences)
            processed_text = filtered_text
            text_lower = filtered_text.lower()

        # 명확한 췌장 낭종 표현 처리 (최우선)
        explicit_cyst_patterns = [
            r'pancreatic\s+cyst',
            r'cyst(?:s)?\s+(?:in|of)\s+(?:the\s+)?pancreas',
            r'cystic\s+lesion(?:s)?\s+(?:in|of)\s+(?:the\s+)?pancreas',
            r'췌장\s*낭종',
            r'낭종.*췌장',
            r'several\s+cysts?.*?pancreas',  # 추가
            r'multiple\s+cysts?.*?pancreas',  # 추가
            r'tiny\s+cysts?.*?pancreas',  # 추가
            r'small\s+cysts?.*?pancreas',  # 추가
        ]

        for pattern in explicit_cyst_patterns:
            processed_text = re.sub(pattern, ' [PANCREATICCYST] ' + pattern, processed_text, flags=re.I)

        # 쉼표 구조 패턴 처리
        comma_structure_pattern = re.compile(
            r'(?:cystic\s+lesions?|cysts?)\s*,\s*(?:pancreas|pancreatic)|(?:several|multiple|few|two|three)\s+(?:cystic\s+lesions?|cysts?)\s*,\s*pancreas',
            re.I
        )

        if comma_structure_pattern.search(text_lower):
            processed_text = re.sub(comma_structure_pattern.pattern, ' [COMMAPATTERN] [PANCREATICCYST] $0', processed_text, flags=re.I)

        # IPMN 패턴 처리
        if 'ipmn' in text_lower or re.search(r'intraductal\s+papillary', text_lower):
            processed_text = re.sub(r'ipmn|intraductal\s+papillary\s+mucinous', ' [IPMN] $0', processed_text, flags=re.I)

        # 췌관 확장 패턴
        processed_text = re.sub(r'p-duct\s+dilatation|pancreatic\s+duct\s+dilatation|duct\s+dilation', ' [PDDUCT] $0', processed_text, flags=re.I)

        # 주췌관 확장 패턴
        if self.pancreatic_duct_dilatation_pattern.search(text_lower):
            processed_text = re.sub(self.pancreatic_duct_dilatation_pattern.pattern, ' [DUCDILATION] $0', processed_text, flags=re.I)

        # 주췌관 확장 + IPMN 패턴
        if self.duct_ipmn_pattern.search(text_lower):
            processed_text = re.sub(self.duct_ipmn_pattern.pattern, ' [DUCDILATION] [IPMN] $0', processed_text, flags=re.I)

        # "or" 패턴 불확실 케이스
        if self.or_pattern.search(text_lower):
            processed_text = re.sub(self.or_pattern.pattern, ' [ORPATTERN] [GRAYZONE] $0', processed_text, flags=re.I)

        # 감별진단(DDx) 패턴
        if self.ddx_pattern.search(text_lower):
            processed_text = re.sub(self.ddx_pattern.pattern, ' [DDXPATTERN] [GRAYZONE] $0', processed_text, flags=re.I)

        # Not delineation 패턴
        if self.not_delineation_pattern.search(text_lower):
            processed_text = re.sub(self.not_delineation_pattern.pattern, ' [NOTDELIN] [GRAYZONE] $0', processed_text, flags=re.I)

        # "No change" + 저음영 병변 패턴
        if self.no_change_low_density_pattern.search(text_lower):
            processed_text = re.sub(self.no_change_low_density_pattern.pattern, ' [NOCHANGE] $0', processed_text, flags=re.I)

        # 양성 저음영 병변 패턴
        if self.benign_low_density_pattern.search(text_lower):
            processed_text = re.sub(self.benign_low_density_pattern.pattern, ' [BENIGNLOW] $0', processed_text, flags=re.I)

        # 다른 장기 낭종 패턴 처리
        processed_text = self._preprocess_other_organ_cysts(processed_text)

        # 잔여 췌장 패턴
        if self.remnant_pattern.search(text_lower):
            processed_text = re.sub(self.remnant_pattern.pattern, ' [REMNANT] $0', processed_text, flags=re.I)

        # 노화 관련 표현
        if self.senile_pattern.search(text_lower):
            processed_text = re.sub(self.senile_pattern.pattern, ' [SENILE] $0', processed_text, flags=re.I)

        # 비낭종 관련 패턴 처리
        processed_text = re.sub(r'\bnormal\s+pancreas', ' [NORMAL] normal pancreas', processed_text, flags=re.I)
        processed_text = re.sub(r'\bunremarkable\s+(?:finding\s+in\s+)?pancreas', ' [NORMAL] unremarkable pancreas', processed_text, flags=re.I)
        processed_text = re.sub(r'\bno\s+abnormality\s+in\s+pancreas', ' [NORMAL] no abnormality in pancreas', processed_text, flags=re.I)
        processed_text = re.sub(r'\bcancer|\btumor|\bmass', ' [TUMOR] $0', processed_text, flags=re.I)
        processed_text = re.sub(r'정상\s*췌장|췌장\s*정상', ' [NORMAL] $0', processed_text, flags=re.I)

        # 부정어 패턴 처리 강화
        for pattern in self.negation_patterns:
            processed_text = re.sub(pattern.pattern, ' [NORMAL] $0', processed_text)

        return processed_text

    def _extract_pattern_features(self, text):
        """패턴 기반 특성 추출"""
        # None 또는 비문자열 입력 처리
        if text is None or not isinstance(text, str):
            text = "" if text is None else str(text)

        text_lower = text.lower()
        features = {}

        # 불확실 케이스 특성 초기화
        features['is_uncertain_case'] = 0

        # 명확한 췌장 낭종 표현 확인 (최우선)
        features['has_definite_pancreatic_cyst'] = 0
        if re.search(r'pancreatic\s+cysts?|cysts?\s+(?:in|of)\s+(?:the\s+)?pancreas|cystic\s+lesions?\s+(?:in|of)\s+(?:the\s+)?pancreas|췌장\s*낭종|낭종.*췌장', text_lower):
            features['has_definite_pancreatic_cyst'] = 1

        # 쉼표 구조 패턴 확인
        features['has_comma_pattern'] = 0
        if re.search(r'(?:cystic\s+lesions?|cysts?)\s*,\s*(?:pancreas|pancreatic)|(?:several|multiple|few|two|three)\s+(?:cystic\s+lesions?|cysts?)\s*,\s*pancreas', text_lower):
            features['has_comma_pattern'] = 1
            features['has_definite_pancreatic_cyst'] = 1  # 쉼표 구조도 명확한 췌장 낭종으로 처리

        # several/multiple + pancreatic cysts 패턴 확인 (추가)
        features['has_several_cysts'] = 0
        if re.search(r'(?:several|multiple|few|two|three)\s+(?:cystic\s+lesions?|cysts?)\s+(?:in|of)\s+(?:the\s+)?pancreas', text_lower):
            features['has_several_cysts'] = 1
            features['has_definite_pancreatic_cyst'] = 1  # 이것도 명확한 췌장 낭종으로 처리

        # IPMN 패턴 확인
        features['has_ipmn'] = 0
        if 'ipmn' in text_lower or re.search(r'(?:main|branch)\s+duct\s+type|intraductal\s+papillary', text_lower):
            features['has_ipmn'] = 1

        # 췌관 확장 패턴 확인
        features['has_duct_dilatation'] = 0
        if re.search(r'p-duct\s+dilatation|pancreatic\s+duct\s+dilatation|duct\s+dilation', text_lower) or self.pancreatic_duct_dilatation_pattern.search(text_lower):
            features['has_duct_dilatation'] = 1

        # 주췌관 확장 + IPMN 패턴 확인
        features['has_duct_ipmn'] = 0
        if self.duct_ipmn_pattern.search(text_lower):
            features['has_duct_ipmn'] = 1
            features['has_ipmn'] = 1  # 관련 특성도 활성화

        # 췌장 낭종 변화 없음 패턴 확인
        features['has_no_change_with_pancreatic_cyst'] = 0
        if re.search(r'no\s+(?:interval\s+)?change\s+of\s+(?:.*?pancreatic\s+cyst|.*?cyst.*?pancreas)', text_lower):
            features['has_no_change_with_pancreatic_cyst'] = 1
            features['has_definite_pancreatic_cyst'] = 1  # 이것도 명확한 췌장 낭종으로 처리

        # "No change" + 저음영 병변 패턴 확인
        features['has_no_change_low_density'] = 0
        if self.no_change_low_density_pattern.search(text_lower):
            features['has_no_change_low_density'] = 1

        # 양성 저음영 병변 패턴 확인
        features['has_benign_low_density'] = 0
        if self.benign_low_density_pattern.search(text_lower):
            # 지방 관련 언급이 있는지 확인
            if re.search(r'(?:fat|fatty|lipoma|infiltration)', text_lower, re.I):
                features['has_benign_low_density'] = 0
                features['is_uncertain_case'] = 1
            else:
                features['has_benign_low_density'] = 1

        # "or" 패턴 불확실 케이스 확인
        features['has_or_pattern'] = 0
        if self.or_pattern.search(text_lower) and re.search(r'pancreas', text_lower, re.I):
            features['has_or_pattern'] = 1
            features['is_uncertain_case'] = 1

        # 감별진단(DDx) 패턴 확인
        features['has_ddx_pattern'] = 0
        if self.ddx_pattern.search(text_lower) and re.search(r'pancreas', text_lower, re.I):
            features['has_ddx_pattern'] = 1
            features['is_uncertain_case'] = 1

        # Not delineation 패턴 확인
        features['has_not_delineation'] = 0
        if self.not_delineation_pattern.search(text_lower):
            features['has_not_delineation'] = 1
            features['is_uncertain_case'] = 1

        # 잔여 췌장 패턴 확인
        features['has_remnant_pancreas'] = 0
        if self.remnant_pattern.search(text_lower):
            features['has_remnant_pancreas'] = 1

        # 정상 췌장 패턴 확인 - 개선된 패턴 인식
        features['has_normal_pancreas'] = 0
        if re.search(r'normal\s+pancreas|unremarkable\s+pancreas|no\s+abnormality\s+in\s+pancreas|정상\s+췌장|췌장\s+정상', text_lower):
            features['has_normal_pancreas'] = 1

        # 부정어 패턴 확인 - 부정어 인식 강화
        features['has_negation'] = 0
        # 부정어 단어 확인
        for term in self.negation_terms:
            if term in text_lower:
                features['has_negation'] = 1
                break
        # 더 복잡한 부정어 패턴 확인
        for pattern in self.negation_patterns:
            if pattern.search(text_lower):
                features['has_negation'] = 1
                break

        # 다른 장기 패턴 확인
        features['has_other_organ'] = 0
        for organ in self.organ_context['negative']:
            if re.search(r'\b' + re.escape(organ.lower()) + r'\b', text_lower):
                features['has_other_organ'] = 1
                break

        # 췌장 언급 확인
        features['has_pancreas_mention'] = 0
        for term in self.organ_context['positive']:
            if term in text_lower:
                features['has_pancreas_mention'] = 1
                break

        # 명확한 췌장 낭종 패턴 확인 (추가)
        features['has_clear_cyst_pattern'] = 0
        for pattern in self.clear_pancreatic_cyst_patterns:
            if pattern.search(text_lower):
                features['has_clear_cyst_pattern'] = 1
                features['has_definite_pancreatic_cyst'] = 1  # 명확한 췌장 낭종으로 처리
                break

        return features

    def _calculate_keyword_score(self, text):
        """키워드 가중치 기반 낭종 점수 계산"""
        # None 또는 비문자열 입력 처리
        if text is None or not isinstance(text, str):
            text = "" if text is None else str(text)

        text_lower = text.lower()
        score = 0.0

        # 명확한 췌장 낭종 표현 처리 (최우선 - 강한 긍정 점수)
        if re.search(r'pancreatic\s+cysts?|cysts?\s+(?:in|of)\s+(?:the\s+)?pancreas|cystic\s+lesions?\s+(?:in|of)\s+(?:the\s+)?pancreas|췌장\s*낭종|낭종.*췌장', text_lower):
            score += 5.0  # 값 증가

        # 쉼표 구조 패턴 확인
        if re.search(r'(?:cystic\s+lesions?|cysts?)\s*,\s*(?:pancreas|pancreatic)|(?:several|multiple|few|two|three)\s+(?:cystic\s+lesions?|cysts?)\s*,\s*pancreas', text_lower):
            score += 5.0  # 값 증가

        # IPMN 관련 패턴 (강한 긍정 점수)
        if 'ipmn' in text_lower or re.search(r'(?:main|branch)\s+duct\s+type|intraductal\s+papillary', text_lower):
            score += 4.5  # 값 증가

        # 췌관 확장 패턴 (긍정 점수)
        if re.search(r'p-duct\s+dilatation|pancreatic\s+duct\s+dilatation|duct\s+dilation', text_lower) or self.pancreatic_duct_dilatation_pattern.search(text_lower):
            score += 3.5  # 값 증가

        # 주췌관 확장 + IPMN 패턴
        if self.duct_ipmn_pattern.search(text_lower):
            score += 4.5  # 값 증가

        # 췌장 낭종 변화 없음 패턴 (강한 긍정 점수)
        if re.search(r'no\s+(?:interval\s+)?change\s+of\s+(?:.*?pancreatic\s+cyst|.*?cyst.*?pancreas)', text_lower):
            score += 4.5  # 값 증가

        # "No change" + 저음영 병변 패턴
        if self.no_change_low_density_pattern.search(text_lower):
            score += 2.5

        # 양성 저음영 병변 패턴
        if self.benign_low_density_pattern.search(text_lower):
            # 지방 관련 언급이 있는지 확인
            if re.search(r'(?:fat|fatty|lipoma|infiltration)', text_lower, re.I):
                score += 0.5  # 불확실 케이스
            else:
                score += 2.5  # 낭종 가능성 더 높음

        # 잔여 췌장 패턴
        if self.remnant_pattern.search(text_lower):
            score += 1.5

        # 노화 관련 표현이 있는 경우 감점
        if self.senile_pattern.search(text_lower):
            score -= 1.5

        # "or" 패턴 불확실 케이스나 DDx 패턴은 중립 점수
        if (self.or_pattern.search(text_lower) or self.ddx_pattern.search(text_lower)) and re.search(r'pancreas', text_lower, re.I):
            score += 0.0  # 중립 점수

        # Not delineation 패턴
        if self.not_delineation_pattern.search(text_lower):
            score += 0.5  # 약한 긍정 점수

        # 부정어 확인 - 부정어 처리 강화
        has_negation = False
        # 단순 부정어 확인
        for term in self.negation_terms:
            if term in text_lower:
                has_negation = True
                break
        # 더 복잡한 부정어 패턴 확인
        for pattern in self.negation_patterns:
            if pattern.search(text_lower):
                has_negation = True
                score -= 5.0  # 부정어 효과 강화
                break

        # 췌장 언급 확인
        is_pancreas_focused = False
        for term in self.organ_context['positive']:
            if term in text_lower:
                is_pancreas_focused = True
                break

        # 다른 장기 언급 확인
        has_other_organ = False
        for term in self.organ_context['negative']:
            if re.search(r'\b' + re.escape(term.lower()) + r'\b', text_lower):
                has_other_organ = True
                break

        # 췌장이 언급되지 않았으나 다른 장기가 언급된 경우의 페널티
        if not is_pancreas_focused and has_other_organ:
            score -= 3.0  # 감점 강화

        # 키워드 가중치 적용
        for keyword, weight in self.keyword_weights.items():
            if keyword in text_lower:
                # 췌장이 언급된 경우 전체 가중치 적용, 그렇지 않으면 감소된 가중치
                if weight > 0 and not is_pancreas_focused:
                    score += weight * 0.2
                else:
                    score += weight

        # 명확한 췌장 낭종 패턴 확인 (추가)
        for pattern in self.clear_pancreatic_cyst_patterns:
            if pattern.search(text_lower):
                score += 5.0  # 매우 높은 점수 부여
                break

        # 부정어가 있는데도 명확한 낭종 패턴이 있는 복잡한 케이스 처리
        if has_negation:
            # "not definite" + cyst 패턴 (현재는 없고 이전에 있었음을 의미)
            if re.search(r'not\s+definite|not\s+detected|not\s+visualized|not\s+seen', text_lower) and re.search(r'cyst|낭종', text_lower):
                score -= 5.0  # 강한 감점 (부정어가 낭종을 직접 부정)

            # "normal pancreas" + cyst 패턴 (이전과 비교 문구)
            elif re.search(r'normal\s+pancreas|unremarkable\s+pancreas', text_lower) and re.search(r'cyst|낭종', text_lower):
                score -= 5.0  # 강한 감점 (정상 췌장이라 명시)

        return score

    def _calculate_adjusted_confidence(self, biobert_prob, pattern_prob, keyword_score, text_lower=""):
        """BioBERT, 패턴 매칭, 키워드 점수에서 조정된 신뢰도 계산"""
        # 기본 앙상블
        base_confidence = self.weight_biobert * biobert_prob + self.weight_pattern * pattern_prob

        # 시그모이드 함수를 사용하여 키워드 점수 적용
        sigmoid_factor = 0.4
        keyword_factor = 1 / (1 + np.exp(-keyword_score * sigmoid_factor))

        # 조정된 신뢰도
        adjusted_confidence = (base_confidence * 0.5 + keyword_factor * 0.5)

        # 명확한 췌장 낭종 표현 확인
        explicit_cyst_patterns = [
            'pancreas cyst', 'pancreatic cyst', 'cyst in pancreas',
            'ipmn', 'p-duct dilatation', 'main duct type',
            'cystic lesion, pancreas', 'cysts, pancreas',
            '췌장 낭종'
        ]

        has_explicit_pancreatic_cyst = any(pattern in text_lower for pattern in explicit_cyst_patterns)

        # 췌장 관련 키워드 확인
        pancreas_keywords = ['pancreas', 'pancreatic', 'uncinate', '췌장', '췌관']
        has_pancreas_mention = any(word in text_lower for word in pancreas_keywords)

        # 명확한 췌장 낭종 언급이 있는 경우의 신뢰도 조정
        if has_explicit_pancreatic_cyst and has_pancreas_mention:
            adjusted_confidence = 0.15  # 낭종 쪽으로 강하게 편향 (0→1 척도에서 0이 낭종)

        # 주췌관 확장 + IPMN 패턴이 있는 경우 낭종 쪽으로 강하게 편향
        if self.duct_ipmn_pattern.search(text_lower):
            adjusted_confidence = 0.15

        # 쉼표 구조 패턴이 있는 경우 낭종 쪽으로 강하게 편향
        if re.search(r'(?:cystic\s+lesions?|cysts?)\s*,\s*(?:pancreas|pancreatic)|(?:several|multiple|few|two|three)\s+(?:cystic\s+lesions?|cysts?)\s*,\s*pancreas', text_lower):
            adjusted_confidence = 0.15

        # Several/Multiple tiny cysts in pancreas 패턴 - 낭종으로 강하게 편향
        if re.search(r'(?:several|multiple|few|two|three)\s+(?:tiny|small)?\s*(?:cystic\s+lesions?|cysts?)\s+(?:in|of)\s+(?:the\s+)?pancreas', text_lower):
            adjusted_confidence = 0.15

        # 부정어 패턴이 있는 경우 비낭종으로 강하게 편향
        for pattern in self.negation_patterns:
            if pattern.search(text_lower):
                adjusted_confidence = 0.85  # 비낭종으로 강하게 편향
                break

        # "or" 패턴 불확실 케이스나 DDx 패턴이 있는 경우 중간값으로 조정
        if ((self.or_pattern.search(text_lower) or self.ddx_pattern.search(text_lower)) and
            re.search(r'pancreas', text_lower, re.I)):
            adjusted_confidence = 0.5  # 불확실로 강제 조정

        # 불확실 범위 조정
        if 0.35 <= adjusted_confidence <= 0.65:
            # 명확한 낭종 언급에 대한 강한 조정
            if has_explicit_pancreatic_cyst:
                adjusted_confidence = 0.15
            # 강한 긍정 키워드 점수에 대한 조정
            elif keyword_score > 2.0:
                adjusted_confidence = 0.25
            # 강한 부정 키워드 점수에 대한 조정
            elif keyword_score < -2.0:
                adjusted_confidence = 0.75

        # 0-1 범위 내로 결과 보장
        return max(0, min(1, adjusted_confidence))

    def get_label_text(self, label_num):
        """숫자 레이블을 정성적 레이블로 변환"""
        if label_num == 0:
            return "PCL"
        elif label_num == 1:
            return "Non-PCL"
        elif label_num == 2:
            return "Uncertain"
        else:
            return "Unknown"

    # 감별진단 컨텍스트 분석 (추가)
    def analyze_differential_diagnosis(self, text):
        """
        감별진단 컨텍스트를 분석하여 췌장 낭종 가능성 평가

        Args:
            text: 분석할 텍스트

        Returns:
            분석 결과 딕셔너리
        """
        result = {
            "has_pancreatic_lesion": False,
            "has_low_density_term": False,
            "has_cyst_mention": False,
            "has_ro_context": False,
            "has_ro_cyst": False,
            "cyst_in_differential": False,
            "classification": "Unknown",
            "confidence": 0.0,
            "evidence_text": ""
        }

        if not isinstance(text, str) or not text:
            return result

        text_lower = text.lower()

        # 1. 췌장 병변 확인
        result["has_pancreatic_lesion"] = any(term in text_lower for term in self.organ_context['positive'])

        if not result["has_pancreatic_lesion"]:
            result["classification"] = "Non-Pancreatic"
            return result

        # 2. 저음영 용어 확인
        low_density_terms = ['low density', 'low attenuation', 'low density lesion', 'low attenuation lesion']
        result["has_low_density_term"] = any(term in text_lower for term in low_density_terms)

        # 3. 낭종 용어 확인
        cyst_terms = ['cyst', 'cystic', 'cystic lesion', 'ipmn', 'intraductal papillary']
        result["has_cyst_mention"] = any(term in text_lower for term in cyst_terms)

        # 4. R/O 컨텍스트 확인 (화살표, R/O, DDx 등)
        ro_patterns = [r'-->', r'--\s+', r'r/o', r'rule out', r'ddx', r'differential', r'vs\.?', r'versus']
        ro_matches = []

        for pattern in ro_patterns:
            matches = list(re.finditer(f'{pattern}(.*?)($|\.|\n)', text_lower))
            ro_matches.extend(matches)

        result["has_ro_context"] = len(ro_matches) > 0

        # 5. R/O 컨텍스트에서 낭종 언급 확인
        if result["has_ro_context"]:
            for match in ro_matches:
                ro_text = match.group(1)
                # 낭종 관련 용어가 감별진단에 포함되어 있는지 확인
                if any(term in ro_text for term in cyst_terms):
                    result["has_ro_cyst"] = True
                    result["cyst_in_differential"] = True
                    result["evidence_text"] = match.group(0)
                    break

        # 6. 부정어 처리 - "No change"는 변화 없음(존재함)으로, "not seen"은 없음으로 처리
        has_no_change = re.search(r'no\s+(?:interval\s+)?change', text_lower) is not None
        has_negation = re.search(r'not\s+(?:definite|detected|visualized|seen)', text_lower) is not None

        # 7. 분류 및 신뢰도 결정
        # 직접적인 낭종 언급 (R/O 컨텍스트 없음)
        if result["has_cyst_mention"] and not result["has_ro_context"] and not has_negation:
            # "No change" 표현은 변화 없음(존재함)
            if has_no_change:
                result["classification"] = "PCL"
                result["confidence"] = 0.9
            elif any(term in text_lower for term in ['definite', 'clear', 'obvious']):
                result["classification"] = "PCL"
                result["confidence"] = 0.9
            else:
                result["classification"] = "Probable PCL"
                result["confidence"] = 0.8

        # R/O 컨텍스트에서 낭종 언급
        elif result["has_ro_cyst"] and not has_negation:
            # vs. 패턴 확인 (낭종 vs. 다른 것)
            if any(vs_pattern in result["evidence_text"].lower() for vs_pattern in ['vs', 'versus']):
                result["classification"] = "Possible PCL"
                result["confidence"] = 0.6
            else:
                result["classification"] = "Probable PCL"
                result["confidence"] = 0.7

        # 직접적인 부정어 + 낭종 언급
        elif has_negation and result["has_cyst_mention"]:
            result["classification"] = "Non-PCL"
            result["confidence"] = 0.8

        # 저음영 병변만 언급 (R/O 컨텍스트 없음)
        elif result["has_low_density_term"] and not result["has_ro_context"]:
            # "No change" 표현은 변화 없음(존재함)
            if has_no_change:
                result["classification"] = "Uncertain"
                result["confidence"] = 0.6
            else:
                result["classification"] = "Uncertain"
                result["confidence"] = 0.3

        # 저음영 병변과 R/O 컨텍스트가 있지만 낭종 언급 없음
        elif result["has_low_density_term"] and result["has_ro_context"]:
            result["classification"] = "Uncertain"
            result["confidence"] = 0.4

        # 기타 췌장 병변
        else:
            result["classification"] = "Non-PCL"
            result["confidence"] = 0.1

        return result

    def predict_with_confidence(self, text):
        """텍스트에 대한 신뢰도를 포함한 예측 - 개선된 패턴 인식"""
        # 췌장 관련 텍스트 추출 - 다른 장기 언급은 이미 제거됨
        focused_text = extract_pancreas_related_text(text)

        # 췌장 관련 텍스트가 없는 경우, 비낭종으로 분류
        if not focused_text.strip():
            return self.NON_PCL_CLASS, 0.90

        text_lower = focused_text.lower()

        # 명확한 췌장 낭종 패턴 확인 - 명확한 패턴이 있으면 바로 낭종으로 분류
        for pattern in self.clear_pancreatic_cyst_patterns:
            if pattern.search(text_lower):
                return self.PCL_CLASS, 0.15  # PCL, 낮은 신뢰도 값 (낭종 확신)

        # 부정어 패턴 확인 - 부정어 패턴이 있으면 바로 비낭종으로 분류
        for pattern in self.negation_patterns:
            if pattern.search(text_lower):
                return self.NON_PCL_CLASS, 0.85  # Non-PCL, 높은 신뢰도 값 (비낭종 확신)

        # IPMN 또는 췌관 확장 관련 패턴 (낭종 가능성 높음)
        ipmn_patterns = ['ipmn', 'branch duct', 'main duct', 'p-duct dilatation', 'intraductal papillary']
        if any(pattern in text_lower for pattern in ipmn_patterns):
            return self.PCL_CLASS, 0.20  # PCL, 낮은 신뢰도 값 (낭종 확신)

        # 주췌관 확장 + IPMN 패턴 확인
        if self.duct_ipmn_pattern.search(text_lower):
            return self.PCL_CLASS, 0.15  # PCL, 낮은 신뢰도 값 (높은 확신)

        # 주췌관 확장 패턴 확인
        if self.pancreatic_duct_dilatation_pattern.search(text_lower):
            # 낭종 관련 명확한 표현이 있는지 확인
            if re.search(r'cyst|cystic|ipmn', text_lower, re.I):
                return self.PCL_CLASS, 0.25  # PCL, 낭종으로 분류
            # 노화 관련 표현이 있는지 확인
            elif self.senile_pattern.search(text_lower):
                return self.NON_PCL_CLASS, 0.7  # 비낭종으로 분류
            else:
                return self.UNCERTAIN_CLASS, 0.5  # 불확실로 분류

        # 쉼표 구조 패턴 확인 (추가 검사)
        if re.search(r'(?:cystic\s+lesions?|cysts?)\s*,\s*(?:pancreas|pancreatic)|(?:several|multiple|few|two|three)\s+(?:cystic\s+lesions?|cysts?)\s*,\s*pancreas', text_lower):
            return self.PCL_CLASS, 0.15  # PCL, 낮은 신뢰도 값 (낭종 확신)

        # 명확한 비낭종 표현 검사
        non_cyst_patterns = [
            'normal pancreas.', 'unremarkable pancreas.', 'no abnormality in pancreas.',
            '정상 췌장', '췌장 정상'
        ]

        if any(pattern in text_lower for pattern in non_cyst_patterns):
            return self.NON_PCL_CLASS, 0.85  # Non-PCL, 높은 신뢰도

        # 양성 저음영 병변 패턴 확인
        if self.benign_low_density_pattern.search(text_lower):
            # 지방 관련 언급이 있는지 확인
            if re.search(r'(?:fat|fatty|lipoma|infiltration)', text_lower, re.I):
                return self.UNCERTAIN_CLASS, 0.5  # 불확실로 분류
            else:
                return self.PCL_CLASS, 0.25  # PCL, 낭종으로 처리

        # "or" 패턴 불확실 케이스 확인
        if self.or_pattern.search(text_lower) and re.search(r'pancreas', text_lower, re.I):
            return self.UNCERTAIN_CLASS, 0.5  # 불확실로 분류

        # 감별진단(DDx) 패턴 확인
        if self.ddx_pattern.search(text_lower) and re.search(r'pancreas', text_lower, re.I):
            return self.UNCERTAIN_CLASS, 0.5  # 불확실로 분류

        # Not delineation 패턴 확인
        if self.not_delineation_pattern.search(text_lower):
            return self.UNCERTAIN_CLASS, 0.5  # 불확실로 분류

        # "No change" + 저음영 병변 패턴 확인
        if self.no_change_low_density_pattern.search(text_lower):
            return self.UNCERTAIN_CLASS, 0.5  # 불확실로 분류

        # 감별진단 컨텍스트 분석 (추가)
        dd_analysis = self.analyze_differential_diagnosis(focused_text)

        # 감별진단 분석에서 "Possible PCL" 등의 분류가 있으면 적용
        if dd_analysis["classification"] in ["PCL", "Probable PCL", "Possible PCL"]:
            if dd_analysis["classification"] == "PCL":
                return self.PCL_CLASS, 0.15
            elif dd_analysis["classification"] == "Probable PCL":
                return self.PCL_CLASS, 0.25
            else:  # "Possible PCL"
                return self.UNCERTAIN_CLASS, 0.5

        # BioBERT + Pattern 분류 앙상블 결과
        # BioBERT용 텍스트 전처리
        processed_text = self._preprocess_text_with_pattern_emphasis(focused_text)

        # BioBERT 예측
        biobert_prob = 0.5  # 기본값은 중립
        if self.biobert_model is not None and self.tokenizer is not None:
            try:
                # 텍스트 인코딩
                encoded = self.tokenizer(processed_text, padding=True, truncation=True, return_tensors="tf")
                # 예측
                logits = self.biobert_model(encoded["input_ids"], encoded["attention_mask"])[0]
                # 확률 계산
                probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]
                # 비낭종 확률
                biobert_prob = probabilities[1]
            except Exception as e:
                print(f"Error in BioBERT prediction: {e}")

        # 패턴 기반 예측
        pattern_prob = 0.5  # 기본값은 중립
        try:
            # 패턴 특성 추출
            features = self._extract_pattern_features(focused_text)

            # 필요한 특성들이 모두 있는지 확인하고 누락된 특성은 0으로 초기화
            required_features = [
                'has_definite_pancreatic_cyst', 'has_comma_pattern', 'has_ipmn',
                'has_duct_dilatation', 'has_duct_ipmn', 'has_no_change_with_pancreatic_cyst',
                'has_no_change_low_density', 'has_benign_low_density', 'has_or_pattern',
                'has_ddx_pattern', 'has_not_delineation', 'has_remnant_pancreas',
                'has_normal_pancreas', 'has_negation', 'has_other_organ',
                'has_pancreas_mention', 'is_uncertain_case', 'has_several_cysts',
                'has_clear_cyst_pattern'
            ]

            # 누락된 특성 추가
            for feat in required_features:
                if feat not in features:
                    features[feat] = 0

            # 특성 벡터 구성
            pattern_features = [features[feat] for feat in sorted(required_features)]

            # 비낭종 확률 예측
            pattern_prob = self.pattern_classifier.predict_proba([pattern_features])[0][1]
        except Exception as e:
            print(f"Error in pattern prediction: {e}")
            import traceback
            traceback.print_exc()

        # 키워드 점수
        keyword_score = self._calculate_keyword_score(focused_text)

        # 최종 신뢰도 계산
        adjusted_confidence = self._calculate_adjusted_confidence(biobert_prob, pattern_prob, keyword_score, text_lower)

        # 임계값 기반 클래스 결정 - 임계값 수정하여 더 많은 낭종을 잡아내도록
        pred_class = self.NON_PCL_CLASS if adjusted_confidence >= self.cyst_threshold else self.PCL_CLASS

        return pred_class, adjusted_confidence

    def predict_with_confidence_and_zone(self, text):
        """신뢰도와 영역 분류를 포함한 예측 - 개선된 패턴 인식"""
        # 췌장 관련 텍스트 먼저 추출 - 다른 장기 언급은 이미 제거됨
        focused_text = extract_pancreas_related_text(text)

        # 췌장 관련 텍스트가 없는 경우 Non-PCL로 바로 분류
        if not focused_text.strip():
            return self.NON_PCL_CLASS, 0.90, self.STANDARD_ZONE

        text_lower = focused_text.lower()

        # 명확한 췌장 낭종 패턴 확인 - 엄격한 패턴 기반 분류 적용
        for pattern in self.clear_pancreatic_cyst_patterns:
            if pattern.search(text_lower):
                return self.PCL_CLASS, 0.15, self.STANDARD_ZONE  # PCL, 낮은 신뢰도 값 (낭종 확신)

        # 부정어 패턴 확인 - 부정어 패턴이 있으면 바로 비낭종으로 분류
        for pattern in self.negation_patterns:
            if pattern.search(text_lower):
                return self.NON_PCL_CLASS, 0.85, self.STANDARD_ZONE  # Non-PCL, 높은 신뢰도 값 (비낭종 확신)

        # IPMN 또는 췌관 확장 관련 패턴 (낭종 가능성 높음)
        ipmn_patterns = ['ipmn', 'branch duct type', 'main duct type', 'intraductal papillary']
        duct_patterns = ['p-duct dilatation', 'pancreatic duct dilatation', 'duct dilation']

        if any(pattern in text_lower for pattern in ipmn_patterns) or any(pattern in text_lower for pattern in duct_patterns):
            return self.PCL_CLASS, 0.20, self.STANDARD_ZONE  # PCL, 낮은 신뢰도 값 (낭종 확신)

        # 주췌관 확장 + IPMN 패턴 확인
        if self.duct_ipmn_pattern.search(text_lower):
            return self.PCL_CLASS, 0.15, self.STANDARD_ZONE  # PCL, 낭종으로 분류 (높은 확신)

        # 주췌관 확장 패턴 확인
        if self.pancreatic_duct_dilatation_pattern.search(text_lower):
            # 낭종 관련 명확한 표현이 있는지 확인
            if re.search(r'cyst|cystic|ipmn', text_lower, re.I):
                return self.PCL_CLASS, 0.25, self.STANDARD_ZONE  # PCL, 낭종으로 분류
            # 노화 관련 표현이 있는지 확인
            elif self.senile_pattern.search(text_lower):
                return self.NON_PCL_CLASS, 0.7, self.STANDARD_ZONE  # 비낭종으로 분류
            else:
                return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE  # 불확실로 분류

        # 전체 문장이 '췌장: 정상' 또는 유사한 표현인 경우
        normal_patterns = [
            'normal pancreas.', 'unremarkable pancreas.', 'no abnormality in pancreas.',
            'pancreas: normal.', 'pancreas: unremarkable.',
            '정상 췌장', '췌장 정상', '췌장: 정상', '췌장에 이상 없음'
        ]

        if any(pattern in text_lower for pattern in normal_patterns):
            return self.NON_PCL_CLASS, 0.80, self.STANDARD_ZONE  # Non-PCL, 높은 확신도

        # 양성 저음영 병변 패턴 확인
        if self.benign_low_density_pattern.search(text_lower):
            # 지방 관련 언급이 있는지 확인
            if re.search(r'(?:fat|fatty|lipoma|infiltration)', text_lower, re.I):
                return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE  # 불확실로 분류
            else:
                return self.PCL_CLASS, 0.25, self.STANDARD_ZONE  # PCL, 낭종으로 처리

        # "or" 패턴 불확실 케이스 확인
        if self.or_pattern.search(text_lower) and re.search(r'pancreas', text_lower, re.I):
            return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE  # 불확실로 분류

        # 감별진단(DDx) 패턴 확인
        if self.ddx_pattern.search(text_lower) and re.search(r'pancreas', text_lower, re.I):
            return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE  # 불확실로 분류

        # Not delineation 패턴 확인
        if self.not_delineation_pattern.search(text_lower):
            return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE  # 불확실로 분류

        # "No change" + 저음영 병변 패턴 확인
        if self.no_change_low_density_pattern.search(text_lower):
            return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE  # 불확실로 분류

        # 지방 침윤 관련 모호한 케이스
        fat_vs_cyst_patterns = [
            'low density lesion', 'low attenuation lesion',
            'fat invagination', 'fatty infiltration', 'fat deposition',
            'r/o cyst vs fat', 'fat vs cyst', 'ddx fat or cyst'
        ]

        has_fat_vs_cyst = any(pattern in text_lower for pattern in fat_vs_cyst_patterns)

        if has_fat_vs_cyst and 'pancreas' in text_lower:
            # 두 가능성이 함께 언급된 경우
            if ('cyst' in text_lower and ('fat' in text_lower or 'fatty' in text_lower)):
                return self.UNCERTAIN_CLASS, 0.80, self.UNCERTAIN_ZONE  # Uncertain, 높은 불확실성

        # 감별진단 컨텍스트 분석 (추가)
        dd_analysis = self.analyze_differential_diagnosis(focused_text)

        # 감별진단 분석에서 "Possible PCL" 등의 분류가 있으면 적용
        if dd_analysis["classification"] in ["PCL", "Probable PCL", "Possible PCL"]:
            if dd_analysis["classification"] == "PCL":
                return self.PCL_CLASS, 0.15, self.STANDARD_ZONE
            elif dd_analysis["classification"] == "Probable PCL":
                return self.PCL_CLASS, 0.25, self.STANDARD_ZONE
            else:  # "Possible PCL"
                return self.UNCERTAIN_CLASS, 0.5, self.UNCERTAIN_ZONE

        # 기본 예측 사용 (BioBERT + 패턴 분류기 앙상블)
        pred_class, confidence = self.predict_with_confidence(focused_text)

        # 신뢰도 기반 불확실 케이스 - 범위 확장 (중간 확신도의 케이스를 더 많이 불확실로 분류)
        if 0.35 <= confidence <= 0.65:  # 범위 확장
            return self.UNCERTAIN_CLASS, confidence, self.UNCERTAIN_ZONE  # Uncertain, 중간 확신도

        # 일반 케이스
        return pred_class, confidence, self.STANDARD_ZONE

    def predict_with_findings_conclusion(self, findings_text, conclusion_text):
        """
        검사결과와 결론을 함께 분석하여 예측 - 다른 장기 언급 완전 제거

        Args:
            findings_text: 검사결과 컬럼 텍스트
            conclusion_text: 결론 컬럼 텍스트

        Returns:
            tuple: (예측 클래스, 신뢰도, 영역)
        """
        # 각 텍스트에서 췌장 관련 부분 추출 - 다른 장기 언급 이미 제거
        findings_focused = extract_pancreas_related_text(findings_text, is_findings=True)
        conclusion_focused = extract_pancreas_related_text(conclusion_text, is_findings=False)

        # 췌장 관련 텍스트가 둘 다 없는 경우 비낭종으로 분류
        if not findings_focused.strip() and not conclusion_focused.strip():
            return self.NON_PCL_CLASS, 0.90, self.STANDARD_ZONE

        # 각 부분 별도 분석
        findings_class, findings_conf, findings_zone = self.predict_with_confidence_and_zone(findings_focused)
        conclusion_class, conclusion_conf, conclusion_zone = self.predict_with_confidence_and_zone(conclusion_focused)

        # 결과 통합 (가중치 적용)
        if findings_class == conclusion_class:  # 일치하면 신뢰도는 높은 쪽 사용
            return findings_class, max(findings_conf, conclusion_conf), findings_zone if findings_class != self.UNCERTAIN_CLASS else self.UNCERTAIN_ZONE
        else:  # 불일치 시 가중치 적용
            # 가중치 적용된 점수 계산
            findings_score = findings_conf * self.findings_weight
            conclusion_score = conclusion_conf * self.conclusion_weight

            # 불확실 클래스가 있으면 불확실로 우선 분류
            if findings_class == self.UNCERTAIN_CLASS or conclusion_class == self.UNCERTAIN_CLASS:
                return self.UNCERTAIN_CLASS, max(findings_conf, conclusion_conf), self.UNCERTAIN_ZONE

            # 명확한 표현 우선 (cystic lesion > low density lesion)
            combined_text = findings_focused + " " + conclusion_focused
            if re.search(r'cystic\s+lesion|cyst|낭종|낭성', combined_text, re.I):
                # 낭종 관련 표현이 결론에 있으면 결론 따름
                if re.search(r'cystic\s+lesion|cyst|낭종|낭성', conclusion_focused, re.I):
                    return self.PCL_CLASS, conclusion_conf, conclusion_zone
                # 낭종 관련 표현이 검사결과에만 있고 결론에 반대 표현이 없으면 검사결과 따름
                elif not re.search(r'no\s+(?:cystic|cyst)|not\s+(?:cystic|cyst)', conclusion_focused, re.I):
                    return self.PCL_CLASS, findings_conf * 0.8, findings_zone  # 약간 신뢰도 감소

            # 주췌관 확장 + IPMN 패턴 확인
            if (self.duct_ipmn_pattern.search(findings_focused) or
                self.duct_ipmn_pattern.search(conclusion_focused)):
                return self.PCL_CLASS, 0.15, self.STANDARD_ZONE  # 낭종으로 분류 (높은 확신)

            # 명확한 췌장 낭종 패턴 확인
            for pattern in self.clear_pancreatic_cyst_patterns:
                if pattern.search(findings_focused) or pattern.search(conclusion_focused):
                    return self.PCL_CLASS, 0.15, self.STANDARD_ZONE

            # 점수 기반 판단
            if findings_score > conclusion_score:
                return findings_class, findings_score, findings_zone
            else:
                return conclusion_class, conclusion_score, conclusion_zone

    def fit(self, texts, labels):
        """하이브리드 모델 학습 - 각 단계별 오류 처리 강화"""
        # 빈 입력 처리
        if not texts or len(texts) == 0:
            print("Error: Empty training texts provided")
            return self

        if not labels or len(labels) == 0:
            print("Error: Empty training labels provided")
            return self

        if len(texts) != len(labels):
            print(f"Error: Number of texts ({len(texts)}) does not match number of labels ({len(labels)})")
            return self

        # 데이터 증강
        print("Augmenting training data...")
        try:
            augmented_texts, augmented_labels = self._augment_training_data(texts, labels)
        except Exception as e:
            print(f"Error in data augmentation: {e}")
            augmented_texts, augmented_labels = texts, labels

        # 췌장 관련 텍스트만 추출 - 다른 장기 언급 제거
        print("Extracting pancreas-related text...")
        try:
            focused_texts = [extract_pancreas_related_text(text) for text in tqdm(augmented_texts)]

            # 다시 한번 다른 장기 언급 제거 (이중 필터링)
            focused_texts = [remove_all_other_organ_mentions(text) for text in tqdm(focused_texts)]
        except Exception as e:
            print(f"Error extracting pancreas text: {e}")
            focused_texts = augmented_texts

        # BioBERT 사용 가능한지 확인
        if self.biobert_model is not None and self.tokenizer is not None:
            # BioBERT 학습 데이터 준비
            print("Preprocessing text...")
            try:
                processed_texts = [self._preprocess_text_with_pattern_emphasis(text) for text in tqdm(focused_texts)]

                print("Encoding data...")
                encoded_texts = self.tokenizer(processed_texts, padding=True, truncation=True, return_tensors="tf")

                # 클래스 가중치 계산
                class_counts = np.bincount(augmented_labels)
                class_weights = class_counts.sum() / (len(class_counts) * class_counts)

                # 낭종 클래스에 추가 가중치 부여
                class_weights[0] *= 1.8  # 1.3에서 1.8로 증가

                # 가중치 적용 손실 함수 사용
                weighted_loss = tf.keras.losses.BinaryCrossentropy(
                    from_logits=True,
                    label_smoothing=0.1  # 레이블 스무딩 추가
                )

                # BioBERT 모델 학습
                print("Training BioBERT model...")
                try:
                    self.biobert_model.compile(
                        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                        loss=weighted_loss,
                        metrics=['accuracy']
                    )

                    self.biobert_model.fit(
                        (encoded_texts["input_ids"], encoded_texts["attention_mask"]),
                        tf.one_hot(augmented_labels, depth=2),
                        epochs=3,
                        batch_size=16,
                        verbose=1
                    )
                except Exception as e:
                    print(f"Error training BioBERT model: {e}")
                    import traceback
                    traceback.print_exc()
                    print("Continuing with pattern-based classifier only...")
            except Exception as e:
                print(f"Error preparing BioBERT data: {e}")
                print("Continuing with pattern-based classifier only...")
        else:
            print("BioBERT model not available. Using pattern-based classifier only.")

        # 패턴 분류기 학습
        print("Training pattern classifier...")
        try:
            # 패턴 분류기에도 클래스 가중치 강화
            self.pattern_classifier = RandomForestClassifier(
                n_estimators=100,
                class_weight={0: 1.8, 1: 0.5},  # 낭종 클래스에 더 높은 가중치 (값 증가)
                random_state=42
            )

            # 각 텍스트에서 특성 추출
            pattern_features = []
            for text in tqdm(focused_texts):
                # 특성 추출
                features = self._extract_pattern_features(text)

                # 필요한 특성들이 모두 있는지 확인하고 누락된 특성은 0으로 초기화
                required_features = [
                    'has_definite_pancreatic_cyst', 'has_comma_pattern', 'has_ipmn',
                    'has_duct_dilatation', 'has_duct_ipmn', 'has_no_change_with_pancreatic_cyst',
                    'has_no_change_low_density', 'has_benign_low_density', 'has_or_pattern',
                    'has_ddx_pattern', 'has_not_delineation', 'has_remnant_pancreas',
                    'has_normal_pancreas', 'has_negation', 'has_other_organ',
                    'has_pancreas_mention', 'is_uncertain_case', 'has_several_cysts',
                    'has_clear_cyst_pattern'
                ]

                # 누락된 특성 추가
                for feat in required_features:
                    if feat not in features:
                        features[feat] = 0

                # 특성 벡터 구성 - 정렬된 순서로
                feature_vector = [features[feat] for feat in sorted(required_features)]
                pattern_features.append(feature_vector)

            # 패턴 분류기 학습
            self.pattern_classifier.fit(pattern_features, augmented_labels)

            # 학습된 특성 중요도 출력
            feature_importance = self.pattern_classifier.feature_importances_
            feature_names = sorted(required_features)

            print("\nPattern Feature Importance:")
            for name, importance in zip(feature_names, feature_importance):
                print(f"{name}: {importance:.4f}")

        except Exception as e:
            print(f"Error training pattern classifier: {e}")
            import traceback
            traceback.print_exc()

        print("Model training complete!")
        return self

    def _augment_training_data(self, texts, labels):
        """발견된 패턴으로 학습 데이터 증강 - 명확한 낭종 패턴 강화"""
        augmented_texts = texts.copy()
        augmented_labels = labels.copy()

        # 각 카테고리의 대표적인 패턴
        typical_patterns = {
            0: [  # 낭종 패턴 - 확장
                "Tiny cystic lesion in the pancreas head.",
                "No significant change of cystic lesion in pancreas.",
                "Several tiny pancreatic cysts, body and tail portions.",
                "Multiple tiny pancreatic cysts.",
                "No interval change of pancreatic cyst.",
                "R/O branch duct type IPMN in pancreas.",
                "A small lobulated cystic lesion in the pancreas.",
                "Several small pancreatic cysts, head and body portion of the pancreas.",
                "Multiple cystic lesions in the pancreas.",
                "No change of two tiny pancreatic cysts, head and body portions.",
                "No interval change of several tiny pancreatic cysts.",
                "Several tiny pancreatic cysts in head, body, and tail portions.",
                "Tiny cyst in pancreas head.",
                "Small cyst in the pancreas.",
                "Pancreatic cyst in the head.",
                "Multiple cysts in the pancreas head and body.",
                # 쉼표 구조 패턴 추가 - 이 패턴 강화
                "Cystic lesion, pancreas.",
                "Cystic lesions, pancreas.",
                "Several cysts, pancreas.",
                "Multiple cysts, pancreas.",
                "Few cysts, pancreas.",
                "Two cysts, pancreas head and body.",
                "Three cystic lesions, pancreas body and tail.",
                # 한글 패턴 추가
                "췌장 낭종",
                "췌장의 낭종성 병변",
                "췌장 체부의 작은 낭종",
                "췌장 두부 및 미부의 다발성 낭종"
            ],
            1: [  # 비낭종 패턴
                "Normal pancreas.",
                "Unremarkable finding in pancreas.",
                "No abnormality in pancreas.",
                "Pancreatic cancer in pancreas head.",
                "Focal fatty infiltration in pancreas.",
                "No evidence of known tiny pancreas cyst...",
                "No definite detectable known tiny pancreas cyst.",
                "No definite abnormality in the pancreas.",
                "정상 췌장",
                "췌장 정상",
                "췌장에 이상 없음"
            ],
            2: [  # 불확실 패턴 추가
                "Benign low density lesion in pancreas, r/o cyst vs. fat.",
                "Low density lesion in pancreas head, ddx) fat or tiny cyst.",
                "Small low attenuation lesion in pancreas, vs. fat deposition.",
                "Not delineation of low density lesion in pancreas.",
                "No change of low density lesion in pancreas.",
                "Benign looking low density lesion, pancreas vs. fat.",
                "DDx) tiny cystic lesion vs. peripancreatic fat.",
                "R/O tiny cyst vs. fatty infiltration.",
                "R/O cyst or lipoma, pancreas."
            ]
        }

        # 명확한 췌장 낭종 패턴 추가 (높은 가중치) - 더 많이 추가
        clear_pancreatic_cyst_patterns = [
            "No change of two tiny pancreatic cysts, head and body portions.",
            "Several tiny pancreatic cysts, body and tail portion of the pancreas.",
            "Multiple cystic lesions in the pancreas.",
            "Several small pancreatic cysts, head and body portion of the pancreas.",
            "No interval change of several tiny pancreatic cysts.",
            "Cystic lesion, pancreas.",
            "Multiple cysts, pancreas.",
            "Several cysts, pancreas.",
            "Multiple cystic lesions, pancreas.",
            "Tiny cysts, pancreas.",
            "Small cystic lesions, pancreas.",
            "Pancreatic cyst in the head.",
            "Pancreatic cyst.",
            "Cyst in pancreas.",
            "췌장 낭종",
            "췌장의 다발성 낭종"
        ]

        # 각 패턴을 10번 추가 (높은 가중치 부여) - 더 많이 추가
        for pattern in clear_pancreatic_cyst_patterns:
            for _ in range(10):  # 5번에서 10번으로 증가
                augmented_texts.append(pattern)
                augmented_labels.append(0)  # 낭종 = 0

        # 부정어 패턴 추가 (높은 가중치) - 추가
        negation_patterns = [
            "Normal pancreas.",
            "Unremarkable pancreas.",
            "No abnormality in the pancreas.",
            "No evidence of pancreatic lesion.",
            "No definite abnormality of the pancreas.",
            "Not visualized pancreatic cyst.",
            "Not definite pancreatic cyst.",
            "정상 췌장",
            "췌장 정상"
        ]

        # 부정어 패턴 추가
        for pattern in negation_patterns:
            for _ in range(8):  # 8번 추가
                augmented_texts.append(pattern)
                augmented_labels.append(1)  # 비낭종 = 1

        # 불확실 케이스 패턴 추가 (높은 가중치)
        uncertain_patterns = [
            "Low density lesion in pancreas, fat vs cyst.",
            "R/O tiny cyst vs fat infiltration in pancreas.",
            "DDx) tiny cystic lesion vs peripancreatic fat.",
            "Not delineation of low density lesion in pancreas.",
            "No change of low density lesion in pancreas.",
            "Benign looking low density lesion, pancreas or fatty infiltration."
        ]

        # 불확실 패턴 추가
        for pattern in uncertain_patterns:
            for _ in range(5):  # 3번에서 5번으로 증가
                augmented_texts.append(pattern)
                augmented_labels.append(2)  # 불확실 = 2

        # 클래스 불균형 처리
        cyst_examples = [text for text, label in zip(texts, labels) if label == 0]
        non_cyst_examples = [text for text, label in zip(texts, labels) if label == 1]
        uncertain_examples = [text for text, label in zip(texts, labels) if label == 2]

        # 낭종 클래스에 더 높은 가중치 부여
        target_count = 100  # 50에서 100으로 증가
        target_class = 0  # 낭종 클래스 증강

        # 대상 클래스 증강
        for i in range(min(int(target_count), 100)):
            # 랜덤 패턴 선택
            pattern = np.random.choice(typical_patterns[target_class])
            augmented_texts.append(pattern)
            augmented_labels.append(target_class)

        # 불확실 클래스도 일부 증강
        for i in range(min(int(target_count/2), 50)):  # 25에서 50으로 증가
            pattern = np.random.choice(typical_patterns[2])
            augmented_texts.append(pattern)
            augmented_labels.append(2)  # 불확실 클래스

        # 오분류 사례 학습 데이터에 추가 (높은 가중치)
        misclassified_cases = [
            "Normal pancreas. Two simple both renal cysts.",
            "No abnormality in pancreas.",
            "Unremarkable finding in pancreas",
            "No definite detectable known tiny pancreas cyst...",
            "No definite detectable tiny pancreatic cyst.",
            "정상 췌장",
            "췌장 정상",
            "췌장에 이상 없음"
        ]

        # 오분류 사례 추가 (비낭종 = 1)
        for case in misclassified_cases:
            # 중복 방지
            if case not in augmented_texts:
                # 8번 추가하여 가중치 증가 (3번에서 8번으로 증가)
                for _ in range(8):
                    augmented_texts.append(case)
                    augmented_labels.append(1)  # 비낭종 = 1

        return augmented_texts, augmented_labels

    def predict(self, texts, findings_texts=None):
        """하이브리드 앙상블을 사용한 여러 텍스트 예측 - 다른 장기 언급 제거 + 오류 처리 강화"""
        # 빈 입력 처리
        if not texts or len(texts) == 0:
            print("Error: Empty prediction texts provided")
            return np.array([]), np.array([]), np.array([])

        predictions = []
        confidences = []
        zones = []  # 회색 영역 분류 추가

        print("Predicting...")
        try:
            # 검사결과와 결론 두 컬럼 모두 제공된 경우
            if findings_texts is not None and len(findings_texts) == len(texts):
                for i, (conclusion_text, findings_text) in enumerate(zip(texts, findings_texts)):
                    try:
                        # 예측 수행
                        pred_class, confidence, zone = self.predict_with_findings_conclusion(
                            findings_text, conclusion_text
                        )
                        predictions.append(pred_class)
                        confidences.append(confidence)
                        zones.append(zone)
                    except Exception as e:
                        print(f"Error predicting text {i}: {e}")
                        # 오류 발생 시 기본값 사용
                        predictions.append(self.NON_PCL_CLASS)  # 기본값은 비낭종
                        confidences.append(0.75)  # 중간 신뢰도
                        zones.append(self.STANDARD_ZONE)  # 일반 영역
            else:
                # 기존 방식 (결론 텍스트만 사용)
                for i, text in enumerate(tqdm(texts)):
                    try:
                        # 예측 수행
                        pred_class, confidence, zone = self.predict_with_confidence_and_zone(text)
                        predictions.append(pred_class)
                        confidences.append(confidence)
                        zones.append(zone)
                    except Exception as e:
                        print(f"Error predicting text {i}: {e}")
                        # 오류 발생 시 기본값 사용
                        predictions.append(self.NON_PCL_CLASS)  # 기본값은 비낭종
                        confidences.append(0.75)  # 중간 신뢰도
                        zones.append(self.STANDARD_ZONE)  # 일반 영역

            # 예측 결과 통계 출력
            total_count = len(predictions)
            pcl_count = sum(1 for p in predictions if p == self.PCL_CLASS)
            non_pcl_count = sum(1 for p in predictions if p == self.NON_PCL_CLASS)
            uncertain_count = sum(1 for p in predictions if p == self.UNCERTAIN_CLASS)

            print(f"\nPrediction Statistics (Total: {total_count}):")
            print(f"PCL (0): {pcl_count} ({pcl_count/total_count*100:.1f}%)")
            print(f"Non-PCL (1): {non_pcl_count} ({non_pcl_count/total_count*100:.1f}%)")
            print(f"Uncertain (2): {uncertain_count} ({uncertain_count/total_count*100:.1f}%)")

            # 영역(zone) 통계 출력
            standard_count = sum(1 for z in zones if z == self.STANDARD_ZONE)
            uncertain_zone_count = sum(1 for z in zones if z == self.UNCERTAIN_ZONE)
            print(f"\nZone Statistics:")
            print(f"Standard: {standard_count} ({standard_count/total_count*100:.1f}%)")
            print(f"Uncertain: {uncertain_zone_count} ({uncertain_zone_count/total_count*100:.1f}%)")

        except Exception as e:
            print(f"Error in prediction process: {e}")
            import traceback
            traceback.print_exc()

        return np.array(predictions), np.array(confidences), np.array(zones)


# ================================================================
# 3. 메인 실행 코드
# ================================================================

def main():
    # 1. 데이터 로드 및 전처리 - 에러 처리 강화
    print("데이터 로드 중...")
    try:
        # 데이터 로드
        cyst_df = read_with_correct_encoding('cyst_training.csv')
        non_cyst_df = read_with_correct_encoding('non_cyst_training.csv')
        test_df = read_with_correct_encoding('unlabeled_test.csv')

        print(f"cyst 데이터 로드: {len(cyst_df)}개 샘플")
        print(f"non-cyst 데이터 로드: {len(non_cyst_df)}개 샘플")
        print(f"테스트 데이터 로드: {len(test_df)}개 샘플")

        # 컬럼명 확인 및 출력
        print(f"cyst 데이터 컬럼: {cyst_df.columns.tolist()}")
        print(f"non-cyst 데이터 컬럼: {non_cyst_df.columns.tolist()}")
        print(f"테스트 데이터 컬럼: {test_df.columns.tolist()}")

    except Exception as e:
        print(f"데이터 로드 오류: {e}")
        print("파일 경로와 이름을 확인하세요.")
        # 오류 발생 시 종료
        import sys
        sys.exit(1)

    # 2. 컬럼 확인 및 추출 - 보다 강력한 처리
    findings_column = '검사결과(text)#7'  # 검사결과 컬럼명
    conclusion_column = '결론및진단#8'  # 결론 컬럼명

    # 각 데이터프레임에서 텍스트 컬럼 확인
    if conclusion_column in cyst_df.columns and conclusion_column in non_cyst_df.columns and conclusion_column in test_df.columns:
        print(f"모든 데이터셋에서 '{conclusion_column}' 컬럼을 찾았습니다.")
    else:
        print(f"경고: 일부 데이터셋에 '{conclusion_column}' 컬럼이 없습니다. 사용 가능한 컬럼을 확인하세요.")
        # 오류 발생 시 종료
        import sys
        sys.exit(1)

    # 검사결과 컬럼 확인
    if findings_column in cyst_df.columns and findings_column in non_cyst_df.columns and findings_column in test_df.columns:
        print(f"모든 데이터셋에서 '{findings_column}' 컬럼을 찾았습니다.")
        use_findings = True
    else:
        # 대체 컬럼명으로 시도 (실제 컬럼명이 다를 수 있음)
        alternative_findings_columns = ['검사결과#7', '검사결과(text)#7', '검사결과']
        for alt_col in alternative_findings_columns:
            if alt_col in cyst_df.columns and alt_col in non_cyst_df.columns and alt_col in test_df.columns:
                findings_column = alt_col
                use_findings = True
                print(f"모든 데이터셋에서 '{findings_column}' 컬럼을 찾았습니다.")
                break
        else:
            print(f"경고: 일부 데이터셋에 검사결과 컬럼이 없습니다. 결론 컬럼만 사용합니다.")
            use_findings = False

    # 3. 학습 데이터 준비 - 텍스트 전처리 강화
    # 결론 텍스트 데이터 추출
    cyst_conclusion_texts = cyst_df[conclusion_column].fillna('').tolist()
    non_cyst_conclusion_texts = non_cyst_df[conclusion_column].fillna('').tolist()
    test_conclusion_texts = test_df[conclusion_column].fillna('').tolist()

    # 원본 텍스트 보관
    original_test_conclusion_texts = test_conclusion_texts.copy()

    # 텍스트 샘플 확인 (한글이 제대로 로드되었는지 검증)
    print("\n결론 텍스트 샘플 확인 (한글 인코딩 검증):")
    for i, text in enumerate(cyst_conclusion_texts[:2] + non_cyst_conclusion_texts[:2] + test_conclusion_texts[:2]):
        print(f"샘플 {i+1}: {text[:100]}...")

    # 검사결과 텍스트 데이터 추출 (사용 가능한 경우)
    if use_findings:
        cyst_findings_texts = cyst_df[findings_column].fillna('').tolist()
        non_cyst_findings_texts = non_cyst_df[findings_column].fillna('').tolist()
        test_findings_texts = test_df[findings_column].fillna('').tolist()

        # 원본 텍스트 보관
        original_test_findings_texts = test_findings_texts.copy()

        # 검사결과 텍스트 샘플 확인
        print("\n검사결과 텍스트 샘플 확인 (한글 인코딩 검증):")
        for i, text in enumerate(cyst_findings_texts[:2] + non_cyst_findings_texts[:2] + test_findings_texts[:2]):
            print(f"샘플 {i+1}: {text[:100]}...")
    else:
        cyst_findings_texts = [''] * len(cyst_conclusion_texts)
        non_cyst_findings_texts = [''] * len(non_cyst_conclusion_texts)
        test_findings_texts = [''] * len(test_conclusion_texts)
        original_test_findings_texts = []

    # 라벨 생성 (0:낭종, 1:비낭종)
    cyst_labels = [0] * len(cyst_conclusion_texts)
    non_cyst_labels = [1] * len(non_cyst_conclusion_texts)

    # 학습 데이터 결합
    train_conclusion_texts = cyst_conclusion_texts + non_cyst_conclusion_texts
    train_findings_texts = cyst_findings_texts + non_cyst_findings_texts
    train_labels = cyst_labels + non_cyst_labels

    print(f"학습 데이터 준비 완료: 낭종({len(cyst_conclusion_texts)}), 비낭종({len(non_cyst_conclusion_texts)})")
    print(f"테스트 데이터: {len(test_conclusion_texts)}개")
    print(f"검사결과 컬럼 사용: {'예' if use_findings else '아니오'}")

    # 4. 모델 초기화 및 학습 - 에러 처리 강화
    print("\n모델 초기화 중...")
    try:
        model = ImprovedPancreaticCystBinaryClassifier(biobert_path="dmis-lab/biobert-v1.1")

        print("\n모델 학습 시작...")
        model.fit(train_conclusion_texts, train_labels)
    except Exception as e:
        print(f"모델 학습 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        print("프로그램을 종료합니다.")
        import sys
        sys.exit(1)

    # 5. 테스트 데이터 예측 및 결과 저장 - 다른 장기 언급 제거 강화
    print("\n테스트 데이터 예측 및 결과 저장 중...")
    try:
        # 다른 장기 언급이 제거된 췌장 관련 텍스트만 추출
        print("췌장 관련 텍스트 추출 중...")
        pancreas_conclusion_texts = [extract_pancreas_related_text(text) for text in tqdm(original_test_conclusion_texts)]

        if use_findings:
            # 검사결과 텍스트도 동일한 방식으로 처리
            pancreas_findings_texts = [extract_pancreas_related_text(text, is_findings=True) for text in tqdm(original_test_findings_texts)]

            # 예측 수행 (두 컬럼 모두 사용)
            print("예측 수행 중...")
            predictions, confidences, zones = model.predict(pancreas_conclusion_texts, pancreas_findings_texts)
        else:
            # 결론 텍스트만 사용하여 예측 수행
            print("예측 수행 중...")
            predictions, confidences, zones = model.predict(pancreas_conclusion_texts)

        # 예측 결과를 DataFrame으로 변환
        print("결과 저장 중...")
        results = {
            'original_conclusion_text': original_test_conclusion_texts,
            'pancreas_conclusion_text': pancreas_conclusion_texts,  # 다른 장기 언급이 제거된 텍스트
            'prediction': predictions,
            'prediction_label': [model.get_label_text(p) for p in predictions],
            'confidence': confidences,
            'zone': zones
        }

        # 검사결과 컬럼도 포함 (사용 가능한 경우)
        if use_findings:
            results['original_findings_text'] = original_test_findings_texts
            results['pancreas_findings_text'] = pancreas_findings_texts  # 다른 장기 언급이 제거된 텍스트

        # 파일 ID 열 추가
        if '변환 ID' in test_df.columns:
            results['file_id'] = test_df['변환 ID'].tolist()

        # DataFrame 생성
        results_df = pd.DataFrame(results)

        # 결과 파일 저장 전 검증
        print("결과 데이터 검증:")
        print(f"총 예측 결과 수: {len(results_df)}")
        print(f"예측 클래스 분포: {results_df['prediction'].value_counts().to_dict()}")

        # 한글 깨짐 없이 저장 (UTF-8 with BOM)
        output_file = 'cyst_predictions_results.csv'
        results_df.to_csv(output_file, index=False, encoding='utf-8-sig')

        # 저장된 파일 확인
        print(f"\n결과가 '{output_file}'에 저장되었습니다.")
        print("저장된 파일의 인코딩 확인:")
        with open(output_file, 'rb') as f:
            raw_data = f.read(10000)
            result = chardet.detect(raw_data)
            print(f"감지된 인코딩: {result['encoding']} (신뢰도: {result['confidence']})")

        # 결과 통계 출력
        total_count = len(predictions)
        pcl_count = sum(1 for p in predictions if p == 0)
        non_pcl_count = sum(1 for p in predictions if p == 1)
        uncertain_count = sum(1 for p in predictions if p == 2)

        print(f"\n예측 결과 통계 (총 {total_count}개):")
        print(f"PCL (0): {pcl_count}개 ({pcl_count/total_count*100:.1f}%)")
        print(f"Non-PCL (1): {non_pcl_count}개 ({non_pcl_count/total_count*100:.1f}%)")
        print(f"Uncertain (2): {uncertain_count}개 ({uncertain_count/total_count*100:.1f}%)")

        # 영역 통계 출력
        standard_count = sum(1 for z in zones if z == "standard")
        uncertain_zone_count = sum(1 for z in zones if z == "uncertain")
        print(f"\nZone Statistics:")
        print(f"Standard: {standard_count}개 ({standard_count/total_count*100:.1f}%)")
        print(f"Uncertain: {uncertain_zone_count}개 ({uncertain_zone_count/total_count*100:.1f}%)")

        # 오분류 의심 사례 분석 (추가)
        # R/O 패턴을 포함하는 텍스트 중 비낭종으로 분류된 케이스 찾기
        ro_pattern = re.compile(r'r/o|vs\.?|versus|ddx', re.I)
        cyst_pattern = re.compile(r'cyst|cystic', re.I)

        potential_misclassifications = []

        for i, (text, pred) in enumerate(zip(pancreas_conclusion_texts, predictions)):
            if not isinstance(text, str) or not text:
                continue

            # R/O 패턴이 있고 그 뒤에 cyst 관련 언급이 있는지 확인
            if ro_pattern.search(text) and cyst_pattern.search(text) and pred == 1:  # 비낭종으로 분류됨
                potential_misclassifications.append({
                    'index': i,
                    'text': text,
                    'prediction': pred,
                    'confidence': confidences[i]
                })

        if potential_misclassifications:
            print(f"\n잠재적 오분류 의심 사례: {len(potential_misclassifications)}개")
            for i, case in enumerate(potential_misclassifications[:5]):  # 처음 5개만 출력
                print(f"\n사례 {i+1}:")
                print(f"텍스트: {case['text']}")
                print(f"예측: {model.get_label_text(case['prediction'])} (신뢰도: {case['confidence']:.2f})")

            # 오분류 의심 사례 별도 저장
            misclass_df = pd.DataFrame(potential_misclassifications)
            misclass_df.to_csv('potential_misclassifications.csv', index=False, encoding='utf-8-sig')
            print(f"\n오분류 의심 사례가 'potential_misclassifications.csv'에 저장되었습니다.")

    except Exception as e:
        print(f"예측 및 결과 저장 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

    print("\n모든 과정이 완료되었습니다!")

# ================================================================
# 4. 개별 텍스트 분석 함수 (디버깅 용도)
# ================================================================

def analyze_text(text, model):
    """개별 텍스트 분석 함수 - 디버깅 용도"""
    print("\n=== 텍스트 분석 ===")
    print(f"원본 텍스트: {text}")

    # 다른 장기 언급 제거
    pancreas_text = extract_pancreas_related_text(text)
    print(f"\n췌장 관련 텍스트: {pancreas_text}")

    # 의학적 맥락 처리
    processed_text = process_medical_context(pancreas_text)
    print(f"\n맥락 처리된 텍스트: {processed_text}")

    # 감별진단 컨텍스트 분석
    dd_analysis = model.analyze_differential_diagnosis(pancreas_text)
    print("\n감별진단 분석 결과:")
    for key, value in dd_analysis.items():
        print(f"  {key}: {value}")

    # 패턴 매칭 분석
    features = model._extract_pattern_features(pancreas_text)
    print("\n패턴 특성:")
    for feat, value in sorted(features.items()):
        print(f"  {feat}: {value}")

    # 키워드 점수 계산
    keyword_score = model._calculate_keyword_score(pancreas_text)
    print(f"\n키워드 점수: {keyword_score:.2f}")

    # 예측 수행
    pred_class, confidence, zone = model.predict_with_confidence_and_zone(pancreas_text)
    print(f"\n예측 결과: {model.get_label_text(pred_class)} (신뢰도: {confidence:.2f}, 영역: {zone})")

    return {
        'original_text': text,
        'pancreas_text': pancreas_text,
        'processed_text': processed_text,
        'dd_analysis': dd_analysis,
        'features': features,
        'keyword_score': keyword_score,
        'prediction': pred_class,
        'prediction_label': model.get_label_text(pred_class),
        'confidence': confidence,
        'zone': zone
    }

# 메인 함수 실행 (프로그램의 시작점)
if __name__ == "__main__":
    main()

데이터 로드 중...
파일 'cyst_training.csv'의 감지된 인코딩: UTF-8-SIG (신뢰도: 1.0)
파일 'non_cyst_training.csv'의 감지된 인코딩: UTF-8-SIG (신뢰도: 1.0)
파일 'unlabeled_test.csv'의 감지된 인코딩: UTF-8-SIG (신뢰도: 1.0)
cyst 데이터 로드: 700개 샘플
non-cyst 데이터 로드: 700개 샘플
테스트 데이터 로드: 886개 샘플
cyst 데이터 컬럼: ['일련번호', '처방일자#3', '검사명#6', '검사결과(text)#7', '결론및진단#8']
non-cyst 데이터 컬럼: ['일련번호', '처방일자#3', '검사명#6', '검사결과(text)#7', '결론및진단#8']
테스트 데이터 컬럼: ['일련번호', '처방일자#3', '검사명#6', '검사결과(text)#7', '결론및진단#8']
모든 데이터셋에서 '결론및진단#8' 컬럼을 찾았습니다.
모든 데이터셋에서 '검사결과(text)#7' 컬럼을 찾았습니다.

결론 텍스트 샘플 확인 (한글 인코딩 검증):
샘플 1: 1. A very tiny cystic lesion in the tail portion of the pancreas, and undetected previously mentione...
샘플 2: 1. No change of a tiny cystic lesion in pancreas head.
     - No delineation of other tiny cystic l...
샘플 3: 1. Normal appearing pancreas.

2. Several tiny hepatic cysts, both.

3. Focal AP shunts, right h...
샘플 4: 1. No detectable pancreas lesion.
2. Probable tiny hepatic hemangioma, S6, no change. 
3. Tiny hepat...
샘플 5: 1. A few tiny

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Error preparing BioBERT data: bad escape \s at position 28
Continuing with pattern-based classifier only...
Training pattern classifier...


100%|██████████| 1860/1860 [00:00<00:00, 12409.18it/s]



Pattern Feature Importance:
has_benign_low_density: 0.0255
has_clear_cyst_pattern: 0.1495
has_comma_pattern: 0.0184
has_ddx_pattern: 0.0000
has_definite_pancreatic_cyst: 0.2294
has_duct_dilatation: 0.0109
has_duct_ipmn: 0.0000
has_ipmn: 0.0130
has_negation: 0.0571
has_no_change_low_density: 0.0147
has_no_change_with_pancreatic_cyst: 0.0245
has_normal_pancreas: 0.1013
has_not_delineation: 0.0128
has_or_pattern: 0.0160
has_other_organ: 0.0000
has_pancreas_mention: 0.2790
has_remnant_pancreas: 0.0019
has_several_cysts: 0.0008
is_uncertain_case: 0.0452
Model training complete!

테스트 데이터 예측 및 결과 저장 중...
췌장 관련 텍스트 추출 중...


100%|██████████| 886/886 [00:00<00:00, 911.54it/s]


예측 수행 중...
Predicting...
Error predicting text 5: bad escape \s at position 28
Error predicting text 8: bad escape \s at position 28
Error predicting text 9: bad escape \s at position 28
Error predicting text 17: bad escape \s at position 28
Error predicting text 24: bad escape \s at position 28
Error predicting text 33: bad escape \s at position 28
Error predicting text 37: bad escape \s at position 28
Error predicting text 53: bad escape \s at position 28
Error predicting text 58: bad escape \s at position 28
Error predicting text 63: bad escape \s at position 28
Error predicting text 66: bad escape \s at position 28
Error predicting text 68: bad escape \s at position 28
Error predicting text 72: bad escape \s at position 28
Error predicting text 76: bad escape \s at position 28
Error predicting text 82: bad escape \s at position 28
Error predicting text 84: bad escape \s at position 28
Error predicting text 85: bad escape \s at position 28
Error predicting text 93: bad escape \s at 